# Import necessary packages:

Read in reviews using BeautifulSoup and then reduced it so we just have the review and the rating. The following code was used to scrape the code: https://gist.github.com/adelweiss/3c13890420662b5041210c48792b4764#file-metacritic-web-scraper . See bottom of the notebook for how the code was edited so the algorithm could be used for my own efficient scraping.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import threading
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Test Using Old vs New Scraper 

We want to look how long it takes to run the scraper

In [ ]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=646dbc256474e5e9c76828cb1e230fd092886970b452363a9feb1ae86b03b889
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


The following fails if we go over too many pages. This line:


*   review.find('div', class_='review_body').find('span').text) 
*   Gives the following error: 'NoneType' object has no attribute 'text'









In [ ]:
review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}

for page in range(0,20):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?page='+str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
                       break 
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text) # if users need to click expand (longer reviews)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text) # for normal sized/shorter reviews  
                                                                                         
    df = pd.DataFrame(review_dict)
    df = df.drop(columns=['name', 'date'])
    df.to_csv(r"drive/My Drive/csv's/Test_old.csv")


AttributeError: ignored

1) Change to code 1 - Successfully removes error and runs for desired number of pages:

In [ ]:
  review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}

for page in range(0,300):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?page='+str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
                       break 
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text 
            if review.find('span', class_='blurb blurb_expanded')
            else None
            )
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text 
                if review.find('div', class_='review_body').find('span') # Adding this if/else removes NoneType error  
                else None # if there is no text attribute do nothing 
                ) 
            
    df = pd.DataFrame(review_dict)
    df = df.drop(columns=['name', 'date'])
    df.to_csv(r"drive/My Drive/csv's/Test_New1.csv")


time: 8min 59s


2) Change to code 2 - Add threads to speed up process of scraping. For the same number of pages, it reduces the time taken from **8 minutes 59 seconds** to **2 minutes 11 seconds**. This **saves 6 minutes 48 seconds**. This reduces the time of scraping by approximately 76%

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(review.find('span', class_='blurb blurb_expanded').text 
            if review.find('span', class_='blurb blurb_expanded')
            else None
            )
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text 
                if review.find('div',class_='review_body').find('span') 
                else None
                )

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,)) # uses the scrap_page method above
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()

df = pd.DataFrame(review_dict)
df = df.drop(columns=['name', 'date'])
df.to_csv(r"drive/My Drive/csv's/Test_New2.csv")


if __name__ == '__main__':
    
    chunkIt(range(300), 100)
    # chunkIt(range(10), 3) ---> [[0, 1, 2], [3, 4, 5], [6, 7, 8, 9]] this would be approx 3 threads
    # https://stackoverflow.com/questions/2130016/splitting-a-list-into-n-parts-of-approximately-equal-length/2130035#2130035 for help

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41
[*

# **Metacritic Datasets:**

# 1) TLOU2 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-of-us-part-ii/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span', class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div', class_='review_body').find('span') else None)

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0                                          #
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df1 = pd.DataFrame(review_dict)
    df1 = df1.drop(columns=['name', 'date'])
    df1.to_csv(r"drive/My Drive/csv's/MetacriticDataset/TLOU2_reviews.csv")


if __name__ == '__main__':
    
    # First argument is the number of pages + 1, 
    # and second argument is the chunk size, so the threads here will be 513 / 100 = 5 ( almost ) 
 
    
    chunkIt(range(512), 100)


[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41
[*

Exception in thread Thread-367:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/u

[*] Working on page: 363
[*] Working on page: 364
[*] Working on page: 365
[*] Working on page: 366
[*] Working on page: 367
[*] Working on page: 368
[*] Working on page: 369
[*] Working on page: 370
[*] Working on page: 371
[*] Working on page: 372
[*] Working on page: 373
[*] Working on page: 374
[*] Working on page: 375
[*] Working on page: 376
[*] Working on page: 377
[*] Working on page: 378
[*] Working on page: 379
[*] Working on page: 380
[*] Working on page: 381
[*] Working on page: 382
[*] Working on page: 383
[*] Working on page: 384
[*] Working on page: 385
[*] Working on page: 386
[*] Working on page: 387
[*] Working on page: 388
[*] Working on page: 389
[*] Working on page: 390
[*] Working on page: 391
[*] Working on page: 392
[*] Working on page: 393
[*] Working on page: 394
[*] Working on page: 395
[*] Working on page: 396
[*] Working on page: 397
[*] Working on page: 398
[*] Working on page: 399
[*] Working on page: 400
[*] Working on page: 401
[*] Working on page: 402


Read in csv and save to dataframe:

In [ ]:
df1 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/TLOU2_reviews.csv",engine='python' )

Drop any null values, as these generate a huge problem with embeddings:

In [ ]:
df1 = df1.dropna()
del df1['Unnamed: 0']
df1 = df1.reset_index(drop=True)

Read to csv to convert the reviews to English in Google Sheets:

In [ ]:
df1.to_csv(r"drive/My Drive/csv's/MetacriticDataset/TLOU2_reviews.csv")

# 2) Pokemon Sword Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/switch/pokemon-sword/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find('span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
      
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()

    df2 = pd.DataFrame(review_dict)
    df2 = df2.drop(columns=['name', 'date'])
    df2.to_csv(r"drive/My Drive/csv's/MetacriticDataset/pokemon_reviews.csv")


if __name__ == '__main__': 
    
    chunkIt(range(27), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26


In [ ]:
df2 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/pokemon_reviews.csv",engine='python' )
df2 = df2.dropna()
del df2['Unnamed: 0']
df2 = df2.reset_index(drop=True)
print(df2)
print(df2['rating'].value_counts())
df2.to_csv(r"drive/My Drive/csv's/MetacriticDataset/pokemon_reviews.csv")

      rating                                             review
0        5.0  My copy of Sword and Shield came in the mail a...
1        3.0  I really wish the games were good. I don't lea...
2        5.0  The game is meh not even counting the Pokemon ...
3        3.0  This is what happen if a company realize that ...
4        4.0  Let us address the elephant in the room first....
...      ...                                                ...
2670     2.0  The game is indeed controversial. There are a ...
2671     5.0  I had a lot of fun with Pokemon Sword and Shie...
2672     5.0  Decepcionante. Esperaba poco del juego después...
2673     5.0  This game is so disappointing and it feels ver...
2674     2.0  Lazy and rushed game for $60\r - National Dex ...

[2675 rows x 2 columns]
0.0     643
10.0    403
1.0     262
4.0     225
3.0     218
2.0     214
9.0     201
8.0     175
5.0     125
6.0     106
7.0     103
Name: rating, dtype: int64


# 3) Modern Warfare Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-modern-warfare/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df3 = pd.DataFrame(review_dict)
    df3 = df3.drop(columns=['name', 'date'])
    df3.to_csv(r"drive/My Drive/csv's/MW_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(42), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41


In [ ]:
df3 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/MW_reviews.csv",engine='python' )
df3 = df3.dropna()
del df3['Unnamed: 0']
df3 = df3.reset_index(drop=True)
print(df3)
print(df3['rating'].value_counts())
df3.to_csv(r"drive/My Drive/csv's/MetacriticDataset/MW_reviews.csv")

      rating                                             review
0       10.0  Playtime 22 days. Its my first cod. Love it. G...
1        0.0  The developers just decided to accuse the Russ...
2        0.0  This game is nothing but a russophobic propaga...
3        0.0  Creators of this game will end like Joseph Goe...
4        0.0  I even created a Metacritic account for the re...
...      ...                                                ...
3985     7.0  The only complaint I have of the game is the s...
3986     0.0  Esse jogo é  maior decepção que eu já tive.  A...
3987    10.0  A pretty good and super fun game. The sounds a...
3988     0.0  This game HAD so much potential. The mechanics...
3989     5.0  Campaign: 8/10, easily the best cod campaign, ...

[3990 rows x 2 columns]
0.0     2037
10.0     818
1.0      369
9.0      188
8.0      146
2.0      120
3.0       72
7.0       71
4.0       71
6.0       50
5.0       48
Name: rating, dtype: int64


# 4) Death Stranding Reviews:


In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/death-stranding/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df4 = pd.DataFrame(review_dict)
    df4 = df4.drop(columns=['name', 'date'])
    df4.to_csv(r"drive/My Drive/csv's/MetacriticDataset/DS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(76), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41
[*

In [ ]:
df4 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/DS_reviews.csv",engine='python' )
df4 = df4.dropna()
del df4['Unnamed: 0']
df4 = df4.reset_index(drop=True)
print(df4)
print(df4['rating'].value_counts())
df4.to_csv(r"drive/My Drive/csv's/MetacriticDataset/DS_reviews.csv")

      rating                                             review
0        2.0  Definitely a waste of $.  Good graphics and od...
1        4.0  Is a very monotonous game,  it has great music...
2        2.0  The trailers for this game sold me the idea of...
3        0.0  If you ever believed there was a god, then you...
4        4.0                 The most boring game in the world 
...      ...                                                ...
7182     9.0  Primero que nada, visualmente el juego es extr...
7183     0.0  I don't know starting from when, the celebrity...
7184     9.0  Amazing game that clearly you love it or you h...
7185     9.0  Every once in a while, there is an specific pi...
7186    10.0  Hekayə 10Atmosfer 10Grafika 10Musiqi 10Hideo K...

[7187 rows x 2 columns]
10.0    3405
0.0     1015
9.0      951
8.0      421
1.0      370
2.0      225
3.0      217
4.0      184
5.0      162
7.0      129
6.0      108
Name: rating, dtype: int64


# 5) Overwatch Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/pc/overwatch/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df5 = pd.DataFrame(review_dict)
    df5 = df5.drop(columns=['name', 'date'])
    df5.to_csv(r"drive/My Drive/csv's/MetacriticDataset/overwatch_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(13), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12


In [ ]:
df5 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/overwatch_reviews.csv",engine='python' )
df5 = df5.dropna()
del df5['Unnamed: 0']
df5 = df5.reset_index(drop=True)
print(df5)
print(df5['rating'].value_counts())
df5.to_csv(r"drive/My Drive/csv's/MetacriticDataset/overwatch_reviews.csv")

      rating                                             review
0        1.0  Now that I have played this game for over ONE ...
1        0.0  Choose a hero from a huge cast of characters!*...
2        0.0  Game is worth a 6 at best. Limited maps. POTG ...
3        0.0  The game is marketed as a game possessing pers...
4        0.0  What happened to Blizzard? It's all style over...
...      ...                                                ...
1286     4.0  The game is good, the idea is interesting, you...
1287    10.0  This game is really epic. I Really recommend t...
1288     0.0  Игра скатилась.В каждой 2-3 катке есть ливер/т...
1289     0.0  I always looked forward to playing Overwatch b...
1290     0.0  Valorant and Overwatch are in a constant gridl...

[1291 rows x 2 columns]
10.0    300
0.0     274
9.0     157
8.0      91
5.0      84
6.0      82
7.0      73
1.0      68
4.0      63
2.0      51
3.0      48
Name: rating, dtype: int64


# 6) Red Dead Redemption 2 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/red-dead-redemption-2/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df6 = pd.DataFrame(review_dict)
    df6 = df6.drop(columns=['name', 'date'])
    df6.to_csv(r"drive/My Drive/csv's/MetacriticDataset/RDR2_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(36), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35


In [ ]:
df6 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/RDR2_reviews.csv",engine='python' )
df6 = df6.dropna()
del df6['Unnamed: 0']
df6 = df6.reset_index(drop=True)
print(df6)
print(df6['rating'].value_counts())
df6.to_csv(r"drive/My Drive/csv's/MetacriticDataset/RDR2_reviews.csv")

      rating                                             review
0        7.0  I'll start off by saying it's impossible to ta...
1        7.0  All the good things you've heard about this ga...
2        7.0  Beautiful graphics, excellent voice acting, lo...
3        7.0  Fair review of RDR2\r I'm almost 15% finished ...
4        8.0  The 10 score reviews here must be some hardcor...
...      ...                                                ...
3333     9.0  Great story ,  and amazing game .. though at t...
3334    10.0  Um dos melhores jogos que já joguei na minha v...
3335     0.0  скучнее только кодзимовский высер.мультиплеер ...
3336    10.0  I spent many hours of my life hunting, fishing...
3337    10.0  Easily one of the greatest gaming experiences ...

[3338 rows x 2 columns]
10.0    1821
9.0      449
8.0      231
0.0      177
7.0      166
5.0      121
6.0      114
4.0       91
1.0       63
3.0       59
2.0       46
Name: rating, dtype: int64


# 7) Minecraft Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/pc/minecraft/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df7 = pd.DataFrame(review_dict)
    df7 = df7.drop(columns=['name', 'date'])
    df7.to_csv(r"drive/My Drive/csv's/MetacriticDataset/minecraft_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(15), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14


In [ ]:
df7 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/minecraft_reviews.csv",engine='python' )
df7 = df7.dropna()
del df7['Unnamed: 0']
df7 = df7.reset_index(drop=True)
print(df7)
print(df7['rating'].value_counts())
df7.to_csv(r"drive/My Drive/csv's/MetacriticDataset/minecraft_reviews.csv")

# 8) Battlefront (2015) Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/star-wars-battlefront/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df8 = pd.DataFrame(review_dict)
    df8 = df8.drop(columns=['name', 'date'])
    df8.to_csv(r"drive/My Drive/csv's/MetacriticDataset/battlefront_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(8), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7


In [ ]:
df8 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/battlefront_reviews.csv",engine='python' )
df8 = df8.dropna()
del df8['Unnamed: 0']
df8 = df8.reset_index(drop=True)
print(df8)
print(df8['rating'].value_counts())
df8.to_csv(r"drive/My Drive/csv's/MetacriticDataset/battlefront_reviews.csv")

     rating                                             review
0       5.0  Looks amazing! Seriously, I mean WOW! You do f...
1       4.0  Good visuals. Gets boring quickly once the nov...
2       3.0  The graphics in this game are absolutely amazi...
3       4.0  While it's true Star Wars fans have been clamo...
4       4.0  Game sounds and looks incredible. thats where ...
..      ...                                                ...
745     6.0  No microtransactions :-). But nothing like the...
746     2.0  Star Wars Battlefront is one of the most beaut...
747     8.0  Отличная игра. Графика для 2015 года просто на...
748     0.0  This game does not even have a bloody single-p...
749     5.0  5/10 - No sé si habrá sido que no entendí la f...

[750 rows x 2 columns]
0.0     118
5.0      85
8.0      77
4.0      75
7.0      71
6.0      63
9.0      58
10.0     54
2.0      52
3.0      50
1.0      47
Name: rating, dtype: int64


# 9) Black Ops 3 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-black-ops-iii/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df9 = pd.DataFrame(review_dict)
    df9 = df9.drop(columns=['name', 'date'])
    df9.to_csv(r"drive/My Drive/csv's/MetacriticDataset/BO3_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(7), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6


In [ ]:
df9 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/BO3_reviews.csv",engine='python' )
df9 = df9.dropna()
del df9['Unnamed: 0']
df9 = df9.reset_index(drop=True)
print(df9)
print(df9['rating'].value_counts())
df9.to_csv(r"drive/My Drive/csv's/MetacriticDataset/BO3_reviews.csv")

     rating                                             review
0         7  Giving this game a zero is silly, BUT giving a...
1         4  I can barley withstand Campaign. Same tricks d...
2         3  Lets look at Halo 5, not the best game in the ...
3         4  Absolutely no innovation and honestly a massiv...
4         8  Black Ops 3 isn't a masterpiece but I believe ...
..      ...                                                ...
611       6  Black ops 3 is a mixed bag for me. On one hand...
612       8  despite being futuristic has no denying that t...
613       6  The solo campaign is absolutely trash, but thi...
614      10  Favorite game of all time. I love everything a...
615       6  okay let me get this over with the campain and...

[616 rows x 2 columns]
0     92
8     82
10    80
7     62
9     57
5     48
6     44
1     44
4     43
3     37
2     27
Name: rating, dtype: int64


# 10) Assassins Creed Odyssey Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/assassins-creed-odyssey/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df10 = pd.DataFrame(review_dict)
    df10 = df10.drop(columns=['name', 'date'])
    df10.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ASO_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(7), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6


In [ ]:
df10 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/ASO_reviews.csv",engine='python' )
df10 = df10.dropna()
del df10['Unnamed: 0']
df10 = df10.reset_index(drop=True)
print(df10)
print(df10['rating'].value_counts())
df10.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ASO_reviews.csv")

     rating                                             review
0      10.0  (I haven't played origins) This Assassin's Cre...
1      10.0  Don't listen to all the ignorant people compla...
2      10.0  I cannot understand the bad reviews for this g...
3      10.0  It doesnt deserve such low score. I played all...
4       5.0  Nah, not a fan of the new formula. Side activi...
..      ...                                                ...
693     9.0  Durée de vie incroyable, graphisme superbe, bo...
694    10.0  I’ve been waiting for AC franchise to come up ...
695     8.0  Assassin's Creed: Odyssey is the best Assassin...
696     8.0  I think this game was the second game of Assas...
697     2.0  It's Origins but more tedious.This is kind of ...

[698 rows x 2 columns]
10.0    184
0.0     105
9.0      92
8.0      77
7.0      51
6.0      39
5.0      39
1.0      34
4.0      34
3.0      25
2.0      18
Name: rating, dtype: int64


# 11) No Man's Sky Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/no-mans-sky/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df11 = pd.DataFrame(review_dict)
    df11 = df11.drop(columns=['name', 'date'])
    df11.to_csv(r"drive/My Drive/csv's/MetacriticDataset/NMS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(14), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


Exception in thread Thread-5659:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.6/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/

[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13


In [ ]:
df11 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/NMS_reviews.csv",engine='python' )
df11 = df11.dropna()
del df11['Unnamed: 0']
df11 = df11.reset_index(drop=True)
print(df11)
print(df11['rating'].value_counts())
df11.to_csv(r"drive/My Drive/csv's/MetacriticDataset/NMS_reviews.csv")

      rating                                             review
0        3.0  No Man's Sky really a different type of game. ...
1        3.0  This game is absolutely massive, and has a dec...
2        7.0  My original review is 2 years old and based of...
3        5.0  NMS can be described as a surprisingly SMALL g...
4        3.0  What appeared to be beautiful at the initial r...
...      ...                                                ...
1186     6.0  It got better, but it is still boring. It's an...
1187     9.0  Es un hermoso y más si amas la astronomía tan ...
1188     0.0  Engañar a la gente con falsas promesas y carac...
1189    10.0  I've been playing this game since day one. At ...
1190     2.0  Has a nice idea, but poorly executed, just fel...

[1191 rows x 2 columns]
10.0    300
0.0     165
9.0     132
8.0     120
3.0      78
5.0      76
1.0      73
4.0      72
6.0      60
7.0      60
2.0      55
Name: rating, dtype: int64


# 12) Halo 5 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/xbox-one/halo-5-guardians/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df12 = pd.DataFrame(review_dict)
    df12 = df12.drop(columns=['name', 'date'])
    df12.to_csv(r"drive/My Drive/csv's/MetacriticDataset/HALO5_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(10), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9


In [ ]:
df12 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/HALO5_reviews.csv",engine='python' )
df12 = df12.dropna()
del df12['Unnamed: 0']
df12 = df12.reset_index(drop=True)
print(df12)
print(df12['rating'].value_counts())
df12.to_csv(r"drive/My Drive/csv's/MetacriticDataset/HALO5_reviews.csv")

     rating                                             review
0       6.0  Halo 5 is all in a polarizing experience. One ...
1       6.0  TL;DR Campaign 4/10 | Mutiplayer 7/10 - - Over...
2       6.0  The worst Halo for me and cannot understand an...
3       7.0  General game play:\r-Movement feels decently f...
4       8.0  I cannot explain how horrendous this game is I...
..      ...                                                ...
945     5.0  Only a few acts as Masterchief, new characters...
946    10.0  It's One of my favorite game, I hope to play t...
947     0.0  Купил Xbox ради этой игры а потом распидарасил...
948     7.0  Si no fuera por la campaña publicitaria algo "...
949     6.0  Siempre pensé que la revolución en Halo llegar...

[950 rows x 2 columns]
10.0    252
9.0     125
8.0     120
0.0     107
7.0      78
6.0      72
4.0      58
5.0      52
3.0      31
1.0      31
2.0      24
Name: rating, dtype: int64


# 13) Borderlands 3 Metacritic

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/pc/borderlands-3/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df13 = pd.DataFrame(review_dict)
    df13 = df13.drop(columns=['name', 'date'])
    df13.to_csv(r"drive/My Drive/csv's/MetacriticDataset/BL3_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(12), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11


In [ ]:
df13 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/BL3_reviews.csv",engine='python' )
df13 = df13.dropna()
del df13['Unnamed: 0']
df13 = df13.reset_index(drop=True)
print(df13)
print(df13['rating'].value_counts())
df13.to_csv(r"drive/My Drive/csv's/MetacriticDataset/BL3_reviews.csv")

      rating                                             review
0        0.0  I personally did not experience any major issu...
1        0.0  What's the point of collecting eridium if it's...
2        1.0  Game is unacceptably buggy as of now. My PC is...
3        0.0  This game has everything to be considered a go...
4        2.0  1080 Ti and cant run normal frames. It micro s...
...      ...                                                ...
1152     4.0  Este jugo es bueno, le pongo esta nota por las...
1153     6.0  Gameplay is great and improved from the pre-qu...
1154     7.0  Es muy buen juego pero cambia tan poco que me ...
1155     0.0  I love Borderlands, I love 2k, Borderlands 3.....
1156     7.0  This game could've had it all: the gameplay, t...

[1157 rows x 2 columns]
10.0    279
0.0     245
9.0     150
8.0     100
4.0      74
1.0      72
2.0      56
5.0      53
7.0      50
3.0      43
6.0      35
Name: rating, dtype: int64


# 14) Assassins Creed Origins Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/assassins-creed-origins/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df14 = pd.DataFrame(review_dict)
    df14 = df14.drop(columns=['name', 'date'])
    df14.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ACOR_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(7), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6


In [ ]:
df14 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/ACOR_reviews.csv",engine='python' )
df14 = df14.dropna()
del df14['Unnamed: 0']
df14 = df14.reset_index(drop=True)
print(df14)
print(df14['rating'].value_counts())
df14.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ACOR_reviews.csv")

     rating                                             review
0       9.0  The best Assassin's Creed game by a mile, and ...
1      10.0  Best Assassins creed game since Black Flag! Lo...
2      10.0  Played for a couple hours I have no issues at ...
3       4.0  HONEST REVIEW: 4/10 mediocre. Honestly I wasn'...
4      10.0  AC origins took a year off of development and ...
..      ...                                                ...
574     7.0  First off, this game looks amazing. The world ...
575     9.0  Tiene una buena trama, es innovador dentro de ...
576    10.0  best assassins creed we had in a while since t...
577    10.0  Ancient Egypt simulator. Absolutely loved it.\...
578     7.0  Good gameplay and visuals, however for an Assa...

[579 rows x 2 columns]
10.0    151
9.0     104
8.0      94
0.0      62
7.0      57
4.0      27
5.0      26
6.0      21
1.0      18
2.0      10
3.0       9
Name: rating, dtype: int64


# 15) Order1886 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-order-1886/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df15 = pd.DataFrame(review_dict)
    df15 = df15.drop(columns=['name', 'date'])
    df15.to_csv(r"drive/My Drive/csv's/MetacriticDataset/1886_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(11), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10


In [ ]:
df15 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/1886_reviews.csv",engine='python' )
df15 = df15.dropna()
del df15['Unnamed: 0']
df15 = df15.reset_index(drop=True)
print(df15)
print(df15['rating'].value_counts())
df15.to_csv(r"drive/My Drive/csv's/MetacriticDataset/1886_reviews.csv")

     rating                                             review
0       6.0  I will start off by saying unfortunately a lot...
1       6.0  i have finished the game on hard difficulty an...
2       6.0  The Order 1886 struggles with finding identity...
3       7.0  This game will put you in a big dilemma. It is...
4       4.0  There is several things to like about the game...
..      ...                                                ...
980     8.0  Great single player game. If you play it for w...
981     4.0  HahahahahahahahahahagahahahahagahahahahahahAha...
982     6.0  Order 1886 was probably intended rather a tech...
983     9.0  Хорошая игра на несколько вечеров. Интересный ...
984    10.0  the first game i played on my Ps4 that i playe...

[985 rows x 2 columns]
10.0    233
8.0     180
9.0     147
7.0      98
6.0      89
5.0      63
0.0      49
4.0      45
2.0      33
1.0      24
3.0      24
Name: rating, dtype: int64


# 16) Days Gone Reviews

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/days-gone/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df16 = pd.DataFrame(review_dict)
    # df16 = df15.drop(columns=['name', 'date'])
    df16.to_csv(r"drive/My Drive/csv's/MetacriticDataset/DG_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(26), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25


In [ ]:
df16 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/DG_reviews.csv",engine='python' )
df16 = df16.dropna()
del df16['Unnamed: 0']
df16 = df16.reset_index(drop=True)
print(df16)
print(df16['rating'].value_counts())
df16.to_csv(r"drive/My Drive/csv's/MetacriticDataset/DG_reviews.csv")

                name  ...                                             review
0         umairbasit  ...  The reviews are so disappointing for this amaz...
1              Colle  ...  I am now 30 hours into the game and I have to ...
2     theydisintegr8  ...  8.5 game for me. Lots of fun and loving the fe...
3          the103don  ...  Having played games since the early 80s I have...
4           Fenrir79  ...  Awesome game. This game feels like a great Joh...
...              ...  ...                                                ...
2459   DennisWaltwin  ...  Jogo com uma bela história e uma trilha sonora...
2460       EderGamer  ...  Muita ação e aventura ao mesmo tempo, sem cont...
2461         GabFire  ...  Apesar do jogo apresentar um monte de problema...
2462          Grippa  ...  hhohohohoohohohhohohohohoohohohohohohohohohoho...
2463       alexslork  ...  DAYS GONE – a breath of fresh air in its genre...

[2464 rows x 4 columns]
10.0    1057
9.0      661
8.0      360
0.0      103

# 17) GTA V Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/grand-theft-auto-v/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df17 = pd.DataFrame(review_dict)
    df17 = df17.drop(columns=['name', 'date'])
    df17.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GTAV_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df17 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/GTAV_reviews.csv",engine='python' )
df17 = df17.dropna()
del df17['Unnamed: 0']
df17 = df17.reset_index(drop=True)
print(df17)
print(df17['rating'].value_counts())
df17.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GTAV_reviews.csv")

     rating                                             review
0       8.0  GTA V was arguably one of the most anticipated...
1      10.0  Small things are important. GTA V has a lot of...
2       7.0  PS4 VERSION:\r Upon first impressions, I thoug...
3      10.0  This is not merely a port , many new things ha...
4       2.0  Im sorry but i cant get over how bad the contr...
..      ...                                                ...
718     0.0  it's the same **** as usual. people are gettin...
719    10.0  10 masterpiece single play game is Masterpiece...
720    10.0  Que se puede decir de este juego? pues eso.......
721     9.0  Aunque la historia no es muy buena el resto de...
722    10.0  GTA 5 - шедевр\rGTA ONLINE - донатерское говно...

[723 rows x 2 columns]
10.0    286
9.0     156
8.0      85
7.0      52
0.0      51
6.0      27
5.0      23
4.0      19
1.0      10
3.0       7
2.0       7
Name: rating, dtype: int64


# 18) Arkham Knight Review:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/batman-arkham-knight/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df18 = pd.DataFrame(review_dict)
    df18 = df18.drop(columns=['name', 'date'])
    df18.to_csv(r"drive/My Drive/csv's/MetacriticDataset/AK_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df18 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/AK_reviews.csv",engine='python' )
df18 = df18.dropna()
del df18['Unnamed: 0']
df18 = df18.reset_index(drop=True)
print(df18)
print(df18['rating'].value_counts())
df18.to_csv(r"drive/My Drive/csv's/MetacriticDataset/AK_reviews.csv")

     rating                                             review
0      10.0  Before I review the game, let me tell you how ...
1       9.0  Ok....so before we proceed; for all those Batm...
2      10.0  I give this game a 10 for the following reason...
3       9.0  Note to everyone who gives this, or most other...
4      10.0  Batman Arkham Knight is a game that improves o...
..      ...                                                ...
745     8.0  Following three fantastic games Arkham Knight,...
746     8.0  damn good game especially the story not really...
747     8.0  Pros:•Good Story•Good Combat•Batmobile•Good Tw...
748     7.0  Big Batman game, but my first Batman game.\r G...
749    10.0  This game is breathtaking, words cannot descri...

[750 rows x 2 columns]
10.0    234
9.0     171
8.0     139
7.0      68
6.0      50
5.0      30
0.0      17
4.0      14
1.0      11
3.0       9
2.0       7
Name: rating, dtype: int64


# 19) Animal Crossing Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/switch/animal-crossing-new-horizons/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df19 = pd.DataFrame(review_dict)
    df19 = df19.drop(columns=['name', 'date'])
    df19.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ACNH_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(37), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36


In [ ]:
df19 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/ACNH_reviews.csv",engine='python' )
df19 = df19.dropna()
del df19['Unnamed: 0']
df19 = df19.reset_index(drop=True)
print(df19)
print(df19['rating'].value_counts())
df19.to_csv(r"drive/My Drive/csv's/MetacriticDataset/ACNH_reviews.csv")

      rating                                             review
0          9  The one island per console is pretty stupid, b...
1          4  "One Dictator, Seven Slaves" same-console mult...
2          4  It’s a fun game, very reminiscent of the old A...
3          1  To actually review this game I think you'll ne...
4          4  I'm cool with the one island per console limit...
...      ...                                                ...
3533       0  Thank you for the drama and deception with my ...
3534       0  Bought this game for the promising coop experi...
3535       0  That finds limitations on the island with resp...
3536       8  I think that journalists too laud Nintendo gam...
3537       2  My boyfriend was super excited for this game, ...

[3538 rows x 2 columns]
0     1334
10     858
9      305
1      300
2      166
8      145
3      113
4      112
5      102
6       55
7       48
Name: rating, dtype: int64


# 20) The Last Guardian review:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-guardian/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df20 = pd.DataFrame(review_dict)
    df20 = df20.drop(columns=['name', 'date'])
    df20.to_csv(r"drive/My Drive/csv's/MetacriticDataset/LG_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(6), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5


In [ ]:
df20 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/LG_reviews.csv",engine='python' )
df20 = df20.dropna()
del df20['Unnamed: 0']
df20 = df20.reset_index(drop=True)
print(df20)
print(df20['rating'].value_counts())
df20.to_csv(r"drive/My Drive/csv's/MetacriticDataset/LG_reviews.csv")

     rating                                             review
0       8.0  If you accept that this game is more of an int...
1       8.0  I think this game is a revelation. There are a...
2       9.0  Es un juego que con todos sus defectos (cámara...
3      10.0  Imprescindible para cualquier persona... creo ...
4      10.0  I love this game.  I know it's not for everyon...
..      ...                                                ...
460    10.0  Perfect game. One of the most intense games I'...
461     9.0  A very special game.\rSome clunky game mechani...
462    10.0  An emotional journey not for everyone. Trico i...
463    10.0  Increíble muy tierno y es buen sucesor  de Sha...
464    10.0  This is a masterpeice that proves video games ...

[465 rows x 2 columns]
10.0    177
9.0      70
8.0      59
7.0      31
0.0      27
6.0      27
5.0      24
2.0      16
3.0      16
1.0       9
4.0       9
Name: rating, dtype: int64


# 21) Resident Evil 3 Reviews:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/resident-evil-3/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df21 = pd.DataFrame(review_dict)
    df21 = df21.drop(columns=['name', 'date'])
    df21.to_csv(r"drive/My Drive/csv's/MetacriticDataset/RE3_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(11), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10


In [ ]:
df21 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/RE3_reviews.csv",engine='python' )
df21 = df21.dropna()
del df21['Unnamed: 0']
df21 = df21.reset_index(drop=True)
print(df21)
print(df21['rating'].value_counts())
df21.to_csv(r"drive/My Drive/csv's/MetacriticDataset/RE3_reviews.csv")

     rating                                             review
0       6.0  Call me delusional, call me nostalgic.. but at...
1       4.0  Proof that better graphics do not make for a b...
2       4.0  Let me be honest on this one, I would love to ...
3       0.0  This is a 5 hours DLC!\rThey tried to compensa...
4       0.0  I just made an account to express how disappoi...
..      ...                                                ...
912     9.0  The feelings I have for this game are very all...
913     8.0  Never got a chance to play the old version of ...
914     6.0  Es juego corto, te toma 4 horas y no es muy re...
915     8.0  Esta entrega sobre todo la saga entera en mi o...
916     5.0  Es un juego con una dinámica de control y unos...

[917 rows x 2 columns]
10.0    182
0.0     120
8.0     110
9.0     109
7.0      77
6.0      71
5.0      65
4.0      65
1.0      50
3.0      36
2.0      32
Name: rating, dtype: int64


# 22) Gears of War 4 Review:

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/xbox-one/gears-of-war-4/user-reviews?page=' + str(
        page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
            # scrap_page(page, review_dict)
        for t in thread_list:
            t.join()
    df22 = pd.DataFrame(review_dict)
    df22 = df22.drop(columns=['name', 'date'])
    df22.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GOW4_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(0,5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df22 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/GOW4_reviews.csv",engine='python' )
df22 = df22.dropna()
del df22['Unnamed: 0']
df22 = df22.reset_index(drop=True)
print(df22)
print(df22['rating'].value_counts())
df22.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GOW4_reviews.csv")

     rating                                             review
0       9.0  Ok so. Firstly i want to address the people gi...
1       9.0  Really not sure what game some of these people...
2      10.0  Are people actually giving this game a rating ...
3       7.0  As expected, the 0 ratings ALL come from peopl...
4       0.0  This was absolute garbage. I refunded this the...
..      ...                                                ...
370    10.0  Excelente juego, un bueb giro, grafica e histo...
371     9.0  Buen juego sin embargo no llena un pequeño vac...
372     8.0  Much like what Halo 4 was to the original tril...
373     9.0  O Jogo tem uma ótima história! Com gráficos im...
374     7.0  another gears game. A 9 for visuals, 6 for sto...

[375 rows x 2 columns]
10.0    136
9.0      67
8.0      60
7.0      32
0.0      25
6.0      15
5.0      15
4.0       7
1.0       6
2.0       6
3.0       6
Name: rating, dtype: int64


# Testing Dataset: Ghost of Tsushima

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/ghost-of-tsushima/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df23 = pd.DataFrame(review_dict)
    df23 = df23.drop(columns=['name', 'date'])
    df23.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GhostT_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(75), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41
[*

In [ ]:
df23 = pd.read_csv("drive/My Drive/csv's/MetacriticDataset/GhostT_reviews.csv",engine='python' )
df23 = df23.dropna()
del df23['Unnamed: 0']
df23 = df23.reset_index(drop=True)
print(df23)
print(df23['rating'].value_counts())
df23.to_csv(r"drive/My Drive/csv's/MetacriticDataset/GhostT_reviews.csv")

      rating                                             review
0       10.0  Amazing to see what Sucker Punch has achieved ...
1       10.0  Very ambitious game from Sucker punch the deve...
2       10.0  The game is purely amazing. Fun combat, a huge...
3       10.0  Absolutely loving this game! Nice and fresh ta...
4       10.0  The level Design os amazing. History and death...
...      ...                                                ...
6466     9.0  Just finished the game, beautiful story, loved...
6467     9.0  ====================IIIIIIIIII GAME RANK : 8.5...
6468     5.0  An absolute blast to play. Suckerpunch has rea...
6469    10.0  This and the Witcher 3 are my top games for th...
6470    10.0  Cool game nice,gameplay,graphics and Japan awe...

[6471 rows x 2 columns]
10.0    4547
9.0     1294
8.0      258
0.0      105
7.0       80
5.0       45
6.0       39
4.0       34
1.0       31
3.0       19
2.0       19
Name: rating, dtype: int64


# **Best 25 PS4 User Reviews:**

The top 25 reviews (that have sufficient reviews) are collected to test for bias:

# 1) Ghost of Tsushima

Already scraped above ^

# 2) Witcher 3: Wild Hunt 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-witcher-3-wild-hunt/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df24 = pd.DataFrame(review_dict)
    df24 = df24.drop(columns=['name', 'date'])
    df24.to_csv(r"drive/My Drive/csv's/Best/W3_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(20), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19


In [ ]:
df24 = pd.read_csv("drive/My Drive/csv's/Best/W3_reviews.csv",engine='python' )
df24 = df24.dropna()
del df24['Unnamed: 0']
df24 = df24.reset_index(drop=True)
print(df24)
print(df24['rating'].value_counts())
df24.to_csv(r"drive/My Drive/csv's/Best/W3_reviews.csv")

      rating                                             review
0       10.0  Very awesome game, reminds me of Red Dead Rede...
1       10.0  This is one of the best RPG and Open World gam...
2       10.0  Amazing game with great gameplay and story. Th...
3       10.0  This will go down as one of the best rpg's eve...
4       10.0  Best RPG game ever ! Large diverse open world ...
...      ...                                                ...
1821    10.0  El mundo del juego es inmersivo, la historia t...
1822     1.0  Купил игру по скидке, думал что понравится так...
1823    10.0  The only thing that needs to be said about thi...
1824    10.0  I played this game five times and planing to m...
1825    10.0  The witcher 3 can easily be the best game of t...

[1826 rows x 2 columns]
10.0    1236
9.0      270
8.0       76
7.0       60
6.0       49
0.0       47
5.0       28
1.0       17
2.0       16
3.0       14
4.0       13
Name: rating, dtype: int64


# 3) The Last of Us Remastered

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-last-of-us-remastered/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df25 = pd.DataFrame(review_dict)
    df25 = df25.drop(columns=['name', 'date'])
    df25.to_csv(r"drive/My Drive/csv's/Best/TLOUR_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(26), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25


In [ ]:
df25 = pd.read_csv("drive/My Drive/csv's/Best/TLOUR_reviews.csv",engine='python' )
df25 = df25.dropna()
del df25['Unnamed: 0']
df25 = df25.reset_index(drop=True)
print(df25)
print(df25['rating'].value_counts())
df25.to_csv(r"drive/My Drive/csv's/Best/TLOUR_reviews.csv")

      rating                                             review
0       10.0  Вместо тысяча слов , я скажу что это лучшая иг...
1       10.0  This game is a 10. I don't believe that a game...
2       10.0  Amazing Game, Perfect history, and a good game...
3       10.0  The Way its meant to be play ...\rThe last of ...
4       10.0  I've been a PC gamer for the majority of my li...
...      ...                                                ...
2410    10.0  I'm super late to the party on this one, I kno...
2411    10.0  Returning to this game hurts because you see h...
2412    10.0  Epic game, a classic masterpiece, a must have ...
2413    10.0  el mejor juego que haya existido 10/10, la his...
2414    10.0  Best game ever you can play pls play this and ...

[2415 rows x 2 columns]
10.0    1733
9.0      347
8.0      122
0.0       62
7.0       45
6.0       26
5.0       23
1.0       19
3.0       15
4.0       14
2.0        9
Name: rating, dtype: int64


# 4) God of War

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/god-of-war/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df26 = pd.DataFrame(review_dict)
    df26 = df26.drop(columns=['name', 'date'])
    df26.to_csv(r"drive/My Drive/csv's/Best/GOW_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(42), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41


In [ ]:
df26 = pd.read_csv("drive/My Drive/csv's/Best/GOW_reviews.csv",engine='python' )
df26 = df26.dropna()
del df26['Unnamed: 0']
df26 = df26.reset_index(drop=True)
print(df26)
print(df26['rating'].value_counts())
df26.to_csv(r"drive/My Drive/csv's/Best/GOW_reviews.csv")

      rating                                             review
0        5.0  As a huge fan of the God of War series and The...
1        4.0  Yes, the graphics are stunning and the storyli...
2        4.0  Overrated but still fun. Just be prepared to g...
3        4.0  Do you like QTE's? How about unskippable cutsc...
4        3.0  I really wanted to like this game...but I don'...
...      ...                                                ...
3663     9.0  This was my first God of War game I have ever ...
3664    10.0                     My favourite game. Masterpiece
3665     9.0  ====================IIIIIIIIII GAME RANK : 9.5...
3666     9.0  The best word to describe this game is "grande...
3667    10.0  This was an excellent reinvention for the God ...

[3668 rows x 2 columns]
10.0    2911
9.0      315
8.0      126
0.0       73
7.0       67
6.0       47
5.0       34
4.0       32
3.0       23
2.0       21
1.0       19
Name: rating, dtype: int64


# 5) SpongeBob SquarePants: Battle for Bikini Bottom - Rehydrated

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/spongebob-squarepants-battle-for-bikini-bottom---rehydrated/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df27 = pd.DataFrame(review_dict)
    df27 = df27.drop(columns=['name', 'date'])
    df27.to_csv(r"drive/My Drive/csv's/Best/SBSP_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(11), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10


In [ ]:
df27 = pd.read_csv("drive/My Drive/csv's/Best/SBSP_reviews.csv",engine='python' )
df27 = df27.dropna()
del df27['Unnamed: 0']
df27 = df27.reset_index(drop=True)
print(df27)
print(df27['rating'].value_counts())
df27.to_csv(r"drive/My Drive/csv's/Best/SBSP_reviews.csv")

     rating                                             review
0      10.0  ‘Spongebob Squarepants: Battle For Bikini Bott...
1      10.0  ntro: Painty the Pirate]\rAre you ready, kids?...
2      10.0  GAMESPOT ПОШЁЛ НАХУЙ!   WE GOT THIS COVERED ПО...
3      10.0  The one of de best's plataform's 3D in 2000's,...
4      10.0  ради того, чтобы оценить этот шедевр, я зареги...
..      ...                                                ...
696     5.0  This is simply a remake of a 2003 game.  it lo...
697    10.0  This game is so fun . And loveable . I don't k...
698    10.0   Определённо уровень Кодзимы,а может даже и выше.
699     9.0  I'm scratching my head wondering if the game I...
700     0.0  great game but too many bugs to care about i a...

[701 rows x 2 columns]
10.0    530
9.0      60
0.0      42
8.0      34
7.0      11
5.0       7
3.0       6
1.0       4
6.0       3
4.0       3
2.0       1
Name: rating, dtype: int64


# 6) Resident Evil 2

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/resident-evil-2/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df28 = pd.DataFrame(review_dict)
    df28 = df28.drop(columns=['name', 'date'])
    df28.to_csv(r"drive/My Drive/csv's/Best/RE2_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df28 = pd.read_csv("drive/My Drive/csv's/Best/RE2_reviews.csv",engine='python' )
df28 = df28.dropna()
del df28['Unnamed: 0']
df28 = df28.reset_index(drop=True)
print(df28)
print(df28['rating'].value_counts())
df28.to_csv(r"drive/My Drive/csv's/Best/RE2_reviews.csv")

     rating                                             review
0      10.0  Omg, this game is absolutely amazing and aweso...
1      10.0  BEST GAME EVER!!!!! Thats all I have got to sa...
2      10.0                                        I loved it.
3      10.0  This has to be the best re-imagining in the se...
4      10.0  One of my favorite game has revived. It is an ...
..      ...                                                ...
704     8.0  Game so far is pretty dope. As in the rest of ...
705     9.0  Es lo mejor que he jugado en PS4, los unicos e...
706    10.0  É um ótimo jogo! Extremamente bonito graficame...
707     8.0  A great remake with engaging gameplay, immersi...
708     9.0  Very good remake. Idea and implementation on h...

[709 rows x 2 columns]
10.0    404
9.0     175
8.0      60
7.0      19
0.0      11
6.0       9
4.0       8
1.0       7
5.0       6
3.0       6
2.0       4
Name: rating, dtype: int64


# 7) Astro Bot: Rescue Mission 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/astro-bot-rescue-mission/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df29 = pd.DataFrame(review_dict)
    df29 = df29.drop(columns=['name', 'date'])
    df29.to_csv(r"drive/My Drive/csv's/Best/AB_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df29 = pd.read_csv("drive/My Drive/csv's/Best/AB_reviews.csv",engine='python' )
df29 = df29.dropna()
del df29['Unnamed: 0']
df29 = df29.reset_index(drop=True)
print(df29)
print(df29['rating'].value_counts())
df29.to_csv(r"drive/My Drive/csv's/Best/AB_reviews.csv")

     rating                                             review
0      10.0  This game is absolutely incredible. Having bee...
1      10.0  Astro Bot Rescue Mission is a superb game; the...
2      10.0  Absolutely fantastic game, this sets a new sta...
3      10.0  This game is phenomenal. I've been looking for...
4      10.0  This game is a must buy! Best looking game for...
..      ...                                                ...
132    10.0  Simplemente maravilloso,una aventura tierna,bu...
133    10.0  One of the best Virtual Reality games I have e...
134    10.0  When RE7 is to scary for you to play in VR tha...
135    10.0  Don't hesitate, just buy it. It's an absolute ...
136     9.0  Best game for VR. Funny, shows different VR po...

[137 rows x 2 columns]
10.0    117
9.0      14
0.0       3
8.0       3
Name: rating, dtype: int64


# 8) NieR: Automata

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/nier-automata/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
       review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df30 = pd.DataFrame(review_dict)
    df30 = df30.drop(columns=['name', 'date'])
    df30.to_csv(r"drive/My Drive/csv's/Best/NA_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df30 = pd.read_csv("drive/My Drive/csv's/Best/NA_reviews.csv",engine='python' )
df30 = df30.dropna()
del df30['Unnamed: 0']
df30 = df30.reset_index(drop=True)
print(df30)
print(df30['rating'].value_counts())
df30.to_csv(r"drive/My Drive/csv's/Best/NA_reviews.csv")

     rating                                             review
0      10.0  A really beautiful game that surpass my expect...
1      10.0  You will have to buy and play this game by you...
2      10.0  THE game that make you CRY after the true ending.
3      10.0  BEST GAME OF THE YEAR. Story, sound, music, an...
4      10.0  NieR: Automata is no doubtedly the most fantas...
..      ...                                                ...
698    10.0  Best game i ever played, the gun goes brrr, th...
699    10.0  An absolute masterpiece, an absolute must-play...
700    10.0  For me is a Beautiful game. I love all about t...
701    10.0  Taro Yoko's amazing story telling, it says it ...
702     9.0  I really enjoyed played this game, I just disa...

[703 rows x 2 columns]
10.0    454
9.0     105
8.0      47
7.0      19
6.0      19
0.0      18
5.0      16
2.0       7
4.0       7
3.0       6
1.0       5
Name: rating, dtype: int64


# 9) Bloodborne 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/bloodborne/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df31 = pd.DataFrame(review_dict)
    df31 = df31.drop(columns=['name', 'date'])
    df31.to_csv(r"drive/My Drive/csv's/Best/BB_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(21), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20


In [ ]:
df31 = pd.read_csv("drive/My Drive/csv's/Best/BB_reviews.csv",engine='python' )
df31 = df31.dropna()
del df31['Unnamed: 0']
df31 = df31.reset_index(drop=True)
print(df31)
print(df31['rating'].value_counts())
df31.to_csv(r"drive/My Drive/csv's/Best/BB_reviews.csv")

      rating                                             review
0       10.0  This is the best game you will get on any next...
1       10.0  Bloodborne is as glorious as it is grotesque, ...
2       10.0  I've never played DS/DS2. That said, this game...
3       10.0  Beautiful game, coming by From Software, I exp...
4       10.0  Do you like the Souls games? Buy this immediat...
...      ...                                                ...
1923    10.0  Um jogo simplesmente incrível, uma evolução id...
1924    10.0  My favourite From game! Takes the formula and ...
1925     0.0  **** this game and **** from software !!!!!!!!...
1926    10.0  This became quickly my all time favorite, and ...
1927    10.0  Um dos melhores jogos de todos os tempos, obri...

[1928 rows x 2 columns]
10.0    1172
9.0      387
8.0      105
0.0       59
7.0       58
6.0       31
5.0       31
1.0       23
4.0       22
3.0       22
2.0       18
Name: rating, dtype: int64


# 10) Detroit: Become Human 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/detroit-become-human/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df32 = pd.DataFrame(review_dict)
    df32 = df32.drop(columns=['name', 'date'])
    df32.to_csv(r"drive/My Drive/csv's/Best/DBH_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(12), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11


In [ ]:
df32 = pd.read_csv("drive/My Drive/csv's/Best/DBH_reviews.csv",engine='python' )
df32 = df32.dropna()
del df32['Unnamed: 0']
df32 = df32.reset_index(drop=True)
print(df32)
print(df32['rating'].value_counts())
df32.to_csv(r"drive/My Drive/csv's/Best/DBH_reviews.csv")

      rating                                             review
0       10.0  Ignore the hate-reviews with a personal agenda...
1       10.0  I'm absolutely blown away by this game. The gr...
2       10.0  I have been playing it since it came out, and ...
3       10.0  I'm a huge fan of David Cage games and I gotta...
4       10.0  The detail, voice acting, the music is incredi...
...      ...                                                ...
1008     2.0  I had to rate a 2 instead of 3 for one reason,...
1009    10.0  El mejor juego que jugué en mi vida, historia ...
1010    10.0  Начнём с хорошего... Графика - супер, захват л...
1011     0.0  Awful, absolutely terrible. This is one of the...
1012     9.0  ====================IIIIIIIIII GAME RANK : 9.0...

[1013 rows x 2 columns]
10.0    572
9.0     217
8.0      96
7.0      41
0.0      26
5.0      18
6.0      17
3.0      10
2.0       6
4.0       5
1.0       5
Name: rating, dtype: int64


# 11) Dark Souls III

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/dark-souls-iii/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df33 = pd.DataFrame(review_dict)
    df33 = df33.drop(columns=['name', 'date'])
    df33.to_csv(r"drive/My Drive/csv's/Best/DS3_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(6), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5


In [ ]:
df33 = pd.read_csv("drive/My Drive/csv's/Best/DS3_reviews.csv",engine='python' )
df33 = df33.dropna()
del df33['Unnamed: 0']
df33 = df33.reset_index(drop=True)
print(df33)
print(df33['rating'].value_counts())
df33.to_csv(r"drive/My Drive/csv's/Best/DS3_reviews.csv")

     rating                                             review
0      10.0  Just started playing this last night, already ...
1       6.0  I'm a diehard Sun Bro and have been Praisin' f...
2       5.0  It's the best game ever since the one that was...
3      10.0  It's really early to say but this game is just...
4      10.0  What an amazing game, 5 hours in, fought that ...
..      ...                                                ...
459     9.0  Wonderful world Wonderful music Wonderful figh...
460    10.0  Моя любимая игра в серии, уступает лишь бладбо...
461     9.0  The best Dark Souls game for me. This game has...
462     8.0  This game didn't instantly grab me, but the mo...
463     9.0  One of the best games ever made.It's include a...

[464 rows x 2 columns]
10.0    198
9.0     130
8.0      53
7.0      25
0.0      19
4.0      10
6.0       9
3.0       7
5.0       6
1.0       4
2.0       3
Name: rating, dtype: int64


# 12) Dreams

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/dreams/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df34 = pd.DataFrame(review_dict)
    df34 = df34.drop(columns=['name', 'date'])
    df34.to_csv(r"drive/My Drive/csv's/Best/DREAMS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(6), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5


In [ ]:
df34 = pd.read_csv("drive/My Drive/csv's/Best/DREAMS_reviews.csv",engine='python' )
df34 = df34.dropna()
del df34['Unnamed: 0']
df34 = df34.reset_index(drop=True)
print(df34)
print(df34['rating'].value_counts())
df34.to_csv(r"drive/My Drive/csv's/Best/DREAMS_reviews.csv")

     rating                                             review
0      10.0  Dreams is an incredible creativity tool, it ha...
1      10.0  Dreams is a definive tool, for making leves, a...
2      10.0  Since day one of early access, I have spent mo...
3      10.0  Been playing consecutively since the first bet...
4      10.0  Incredible game. Amazing campaign. Outstanding...
..      ...                                                ...
472     9.0  Es el mejor juego de la historia de ps4. La hi...
473     8.0  I am happy that dreams does exist, It is uniqu...
474     9.0  Very good concept. Really enjoyed using my ima...
475    10.0  This game has Tower Ball! This is the best gam...
476    10.0  Iuuuuibfb Betty ven jakbwbwbwddbwbwansnbsnsbsb...

[477 rows x 2 columns]
10.0    380
9.0      39
1.0       8
5.0       8
0.0       8
7.0       8
8.0       7
4.0       6
6.0       6
3.0       4
2.0       3
Name: rating, dtype: int64


# 13) Marvel's Spider-Man

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/marvels-spider-man/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df35 = pd.DataFrame(review_dict)
    df35 = df35.drop(columns=['name', 'date'])
    df35.to_csv(r"drive/My Drive/csv's/Best/SM_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(17), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16


In [ ]:
df35 = pd.read_csv("drive/My Drive/csv's/Best/SM_reviews.csv",engine='python' )
df35 = df35.dropna()
del df35['Unnamed: 0']
df35 = df35.reset_index(drop=True)
print(df35)
print(df35['rating'].value_counts())
df35.to_csv(r"drive/My Drive/csv's/Best/SM_reviews.csv")

      rating                                             review
0        8.0  Это лучшая игра про человека-паука , поиграйте...
1        9.0  Great gameplay, great graphics,  good story an...
2       10.0  Pros: Gorgeous game especially on the pro. Dra...
3       10.0  What a beautifully crafted game, graphics, gam...
4       10.0  I'm not an avid gamer and this was my first ti...
...      ...                                                ...
1540    10.0  Spider-Man is a splendidly tuned action game w...
1541    10.0  No sloilers! I like games with open world (min...
1542    10.0  Awesome game! Fun combat, amazing web slinging...
1543     9.0  good. veery good. maybe even the best of all g...
1544     9.0  Very good game, graphics, feelings, content. M...

[1545 rows x 2 columns]
10.0    690
9.0     475
8.0     194
7.0      77
6.0      26
0.0      25
5.0      20
4.0      13
3.0      12
1.0       7
2.0       6
Name: rating, dtype: int64


# 14) Dragon Quest XI

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/dragon-quest-xi-echoes-of-an-elusive-age/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df36 = pd.DataFrame(review_dict)
    df36 = df36.drop(columns=['name', 'date'])
    df36.to_csv(r"drive/My Drive/csv's/Best/DQ_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df36 = pd.read_csv("drive/My Drive/csv's/Best/DQ_reviews.csv",engine='python' )
df36 = df36.dropna()
del df36['Unnamed: 0']
df36 = df36.reset_index(drop=True)
print(df36)
print(df36['rating'].value_counts())
df36.to_csv(r"drive/My Drive/csv's/Best/DQ_reviews.csv")

     rating                                             review
0      10.0  Last year the best JRPG on the PS4 was Persona...
1      10.0  great and classic game I love it so much.this ...
2      10.0  Dragon Quest XI is one of the best video games...
3      10.0  I was expecting a good JRPG, that would bring ...
4      10.0  at last the real rpg game whe deserve, dragon ...
..      ...                                                ...
180     8.0  I give it a 8 only because the music wasn’t th...
181     9.0  Really great game. Does everything you ask for...
182    10.0  Nowadays, while RPG now shifting toward action...
183     6.0  This is one of those games that I loved the fi...
184    10.0  It’s my first dq game and im love it the story...

[185 rows x 2 columns]
10.0    96
9.0     36
8.0     16
7.0     13
0.0      8
5.0      5
6.0      4
4.0      3
2.0      2
3.0      1
1.0      1
Name: rating, dtype: int64


# 15) Persona 5

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/persona-5/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df37 = pd.DataFrame(review_dict)
    df37 = df37.drop(columns=['name', 'date'])
    df37.to_csv(r"drive/My Drive/csv's/Best/P5_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(10), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9


In [ ]:
df37 = pd.read_csv("drive/My Drive/csv's/Best/P5_reviews.csv",engine='python' )
df37 = df37.dropna()
del df37['Unnamed: 0']
df37 = df37.reset_index(drop=True)
print(df37)
print(df37['rating'].value_counts())
df37.to_csv(r"drive/My Drive/csv's/Best/P5_reviews.csv")

     rating                                             review
0       9.0  A great game. I loved P3 and P4 maybe a bit mo...
1       9.0  Game could easily be a 10 but i don't think an...
2       9.0  Best Persona game yet. I have played most of A...
3      10.0  The game is a masterpiece of JRPG goodness. It...
4       9.0  This is my first entry into the series. 113 ho...
..      ...                                                ...
892     9.0  Actual Score: 9.5/10Considering I’ve been unab...
893    10.0  Oyandigim en iyi oyun, herhangi weeb oyunu buu...
894     8.0  Good game. Good game. Good game. Good game. Go...
895     8.0  Persona 5 is amazing but it's also extremely f...
896     7.0  Finished the game for almost 250hrs. I dont li...

[897 rows x 2 columns]
10.0    622
9.0     125
8.0      47
7.0      35
6.0      15
0.0      14
1.0      11
3.0       9
5.0       9
4.0       6
2.0       4
Name: rating, dtype: int64


# 16) Life is Strange

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/life-is-strange/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find('span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df38 = pd.DataFrame(review_dict)
    df38 = df38.drop(columns=['name', 'date'])
    df38.to_csv(r"drive/My Drive/csv's/Best/LIS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df38 = pd.read_csv("drive/My Drive/csv's/Best/LIS_reviews.csv",engine='python' )
df38 = df38.dropna()
del df38['Unnamed: 0']
df38 = df38.reset_index(drop=True)
print(df38)
print(df38['rating'].value_counts())
df38.to_csv(r"drive/My Drive/csv's/Best/LIS_reviews.csv")

     rating                                             review
0      10.0  Further proof that video games can tell emotio...
1       5.0  La idea de la historia y del juego esta bien, ...
2       8.0                           Deutsche Review - Kritik
3       7.0  An enjoyable teen drama. With its likable char...
4       5.0  I know people will disagree with me but honest...
..      ...                                                ...
241    10.0  I really liked the Story. The worldbuilding wo...
242    10.0  Life Is Strange is the best game ever, DONTNOD...
243    10.0  Maravilloso de principio a fin... Se disfruta ...
244    10.0  Ótimo jogo,traz uma experiência única,nós faz ...
245     9.0  Great game what i love about this game is u ca...

[246 rows x 2 columns]
10.0    101
9.0      59
8.0      34
7.0      28
4.0       6
6.0       6
5.0       5
1.0       2
2.0       2
3.0       2
0.0       1
Name: rating, dtype: int64


# 17) A Plague Tale: Innocence 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/a-plague-tale-innocence/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df39 = pd.DataFrame(review_dict)
    df39 = df39.drop(columns=['name', 'date'])
    df39.to_csv(r"drive/My Drive/csv's/Best/PTI_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df39 = pd.read_csv("drive/My Drive/csv's/Best/PTI_reviews.csv",engine='python' )
df39 = df39.dropna()
del df39['Unnamed: 0']
df39 = df39.reset_index(drop=True)
print(df39)
print(df39['rating'].value_counts())
df39.to_csv(r"drive/My Drive/csv's/Best/PTI_reviews.csv")

     rating                                             review
0      10.0  What can I say? Wow! I found out about this ga...
1       9.0  Awesome and very Unique game with outstanding ...
2       9.0  The game is beautiful and amazing it is a stor...
3       7.0  Giving it a 7 as a sign of support. If you lik...
4       6.0  I can see the developers put a LOT of love in ...
..      ...                                                ...
177     0.0  je trouve que tout dans ce jeux est bon, beau,...
178     9.0  excelente juego no le tenia mucha fe, pero a l...
179     9.0  Ce jeu est tellement bon que j'ai du m'imposer...
180    10.0  This game is masterpiece . And most under rate...
181    10.0  This game is masterpiece made by asobo studios...

[182 rows x 2 columns]
10.0    68
9.0     53
8.0     25
7.0     12
6.0     11
5.0      4
0.0      3
1.0      2
3.0      2
2.0      1
4.0      1
Name: rating, dtype: int64


# 18) The Evil Within 2

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/the-evil-within-2/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df40 = pd.DataFrame(review_dict)
    df40 = df40.drop(columns=['name', 'date'])
    df40.to_csv(r"drive/My Drive/csv's/Best/EW2_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df40 = pd.read_csv("drive/My Drive/csv's/Best/EW2_reviews.csv",engine='python' )
df40 = df40.dropna()
del df40['Unnamed: 0']
df40 = df40.reset_index(drop=True)
print(df40)
print(df40['rating'].value_counts())
df40.to_csv(r"drive/My Drive/csv's/Best/EW2_reviews.csv")

     rating                                             review
0       9.0  That's a very good second entrance from Tango ...
1      10.0  this game is big improvement about anything\rt...
2       9.0  Really good game, I was slightly worried when ...
3      10.0  Wow, this game is amazing and incredible horro...
4       7.0  The Evil Within 2 is a slightly above average ...
..      ...                                                ...
194     8.0  Una digna secuela de una gran primera parte, a...
195     8.0  Um excelente jogo, possui uma história envolve...
196     8.0  The graphics and gameplay are awesome, the sem...
197    10.0  While The Evil Within 2 isn’t particularly sca...
198    10.0  Super nice game from one of my favorite Horror...

[199 rows x 2 columns]
10.0    62
8.0     49
9.0     45
7.0     12
0.0      7
4.0      7
6.0      6
3.0      4
5.0      4
2.0      2
1.0      1
Name: rating, dtype: int64


# 19) Titanfall 2

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/titanfall-2/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df41 = pd.DataFrame(review_dict)
    df41 = df41.drop(columns=['name', 'date'])
    df41.to_csv(r"drive/My Drive/csv's/Best/TF2_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df41 = pd.read_csv("drive/My Drive/csv's/Best/TF2_reviews.csv",engine='python' )
df41 = df41.dropna()
del df41['Unnamed: 0']
df41 = df41.reset_index(drop=True)
print(df41)
print(df41['rating'].value_counts())
df41.to_csv(r"drive/My Drive/csv's/Best/TF2_reviews.csv")

     rating                                             review
0       9.0  Titanfall 2 has really surprised me. I was not...
1       9.0  So far the campaign is pretty fun to play. It'...
2      10.0  Simply the best. Fast, fun and furious. PS pla...
3      10.0  Titanfall 2 is the most exciting shooter I hav...
4      10.0  Fluid that's all that needs to be said about T...
..      ...                                                ...
347     7.0  The time I this game for first time it was har...
348    10.0  Unlike al other fps shooters at the time and e...
349    10.0  The story is great! And the movment too! Every...
350     9.0  Where do I start?Campaign:If you're looking fo...
351     0.0  The greatest FPS solo campaign I've ever playe...

[352 rows x 2 columns]
10.0    141
9.0     122
8.0      44
7.0      20
0.0       5
6.0       5
2.0       3
3.0       3
4.0       3
1.0       3
5.0       3
Name: rating, dtype: int64


# 20) Uncharted 4: A Thief's End 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/uncharted-4-a-thiefs-end/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df42 = pd.DataFrame(review_dict)
    df42 = df42.drop(columns=['name', 'date'])
    df42.to_csv(r"drive/My Drive/csv's/Best/UC4_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(23), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22


In [ ]:
df42 = pd.read_csv("drive/My Drive/csv's/Best/UC4_reviews.csv",engine='python' )
df42 = df42.dropna()
del df42['Unnamed: 0']
df42 = df42.reset_index(drop=True)
print(df42)
print(df42['rating'].value_counts())
df42.to_csv(r"drive/My Drive/csv's/Best/UC4_reviews.csv")

      rating                                             review
0       10.0  From Beginning to end Uncharted 4: A thief's e...
1       10.0  Naughty Dog delivers again.  I played most of ...
2       10.0  Damn, Damn, Damn what a game I love it, I love...
3       10.0  The best of Uncharted franchise. The graphics ...
4       10.0  you really no they are quicly made accounts ju...
...      ...                                                ...
2189    10.0  El juego increíble. Una pena que no lo pueda p...
2190     8.0  Simple story, complex characters is essentiall...
2191     8.0  Muy buen juego, mejor que los anteriores, graf...
2192    10.0  Fechou a série com chave de ouro, foi um dos m...
2193     9.0  ====================IIIIIIIIII GAME RANK : 9.0...

[2194 rows x 2 columns]
10.0    1395
9.0      362
8.0      161
7.0      113
6.0       43
0.0       35
5.0       29
4.0       25
1.0       16
3.0       10
2.0        5
Name: rating, dtype: int64


# 21) Rachet & Clank

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/ratchet-clank/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df43 = pd.DataFrame(review_dict)
    df43 = df43.drop(columns=['name', 'date'])
    df43.to_csv(r"drive/My Drive/csv's/Best/RC_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df43 = pd.read_csv("drive/My Drive/csv's/Best/RC_reviews.csv",engine='python' )
df43 = df43.dropna()
del df43['Unnamed: 0']
df43 = df43.reset_index(drop=True)
print(df43)
print(df43['rating'].value_counts())
df43.to_csv(r"drive/My Drive/csv's/Best/RC_reviews.csv")

     rating                                             review
0       9.0  So, i'm a newcomer. Never got to experience pl...
1      10.0  FANTASTIC game - it is so refreshing to see an...
2      10.0  Okay, so initially I wasn't really excited for...
3      10.0  this Ratchet and clank game is one of the best...
4      10.0  I love this series, so I was thrilled to have ...
..      ...                                                ...
358     9.0  I loved the game from beginning to end. The gr...
359     8.0  First game where 30 fps doesn't feel clunky, s...
360     8.0  Ratchet and Clank is a culmination of everythi...
361     8.0  the game is a good enought retelling but the c...
362     8.0  Phenomenal all around game! Upgrade system of ...

[363 rows x 2 columns]
10.0    128
9.0     107
8.0      75
7.0      27
5.0      11
6.0       7
1.0       3
4.0       3
2.0       2
Name: rating, dtype: int64


# 22) Uncharted: The Nathan Drake Collection 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/uncharted-the-nathan-drake-collection/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df44 = pd.DataFrame(review_dict)
    df44 = df44.drop(columns=['name', 'date'])
    df44.to_csv(r"drive/My Drive/csv's/Best/UCC_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df44 = pd.read_csv("drive/My Drive/csv's/Best/UCC_reviews.csv",engine='python' )
df44 = df44.dropna()
del df44['Unnamed: 0']
df44 = df44.reset_index(drop=True)
print(df44)
print(df44['rating'].value_counts())
df44.to_csv(r"drive/My Drive/csv's/Best/UCC_reviews.csv")

     rating                                             review
0      10.0  Feeling jealous listening my PS3 friends talki...
1      10.0  It's funny to see how poor are these xbox/nint...
2      10.0  It's brilliant that Bluepoint remastered these...
3      10.0  I don't wanna say long. Why? I feel I don't ha...
4      10.0  3 Great games looking better than ever, especi...
..      ...                                                ...
333    10.0  Отличная серия. Самая лучшая как по мне 3 част...
334    10.0              It's a  very injoyable set of game's.
335    10.0  Feels like playing a mix of Mission Impossible...
336     9.0  Just created my account and wanted to come rev...
337     7.0  I never played the Uncharted Series until well...

[338 rows x 2 columns]
10.0    133
9.0      85
8.0      56
7.0      15
0.0      14
6.0      12
5.0      11
4.0       8
1.0       2
2.0       1
3.0       1
Name: rating, dtype: int64


# 23) Rocket League 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/rocket-league/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df45 = pd.DataFrame(review_dict)
    df45 = df45.drop(columns=['name', 'date'])
    df45.to_csv(r"drive/My Drive/csv's/Best/RL_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df45 = pd.read_csv("drive/My Drive/csv's/Best/RL_reviews.csv",engine='python' )
df45 = df45.dropna()
del df45['Unnamed: 0']
df45 = df45.reset_index(drop=True)
print(df45)
print(df45['rating'].value_counts())
df45.to_csv(r"drive/My Drive/csv's/Best/RL_reviews.csv")

     rating                                             review
0      10.0  An incredible game created by incredible devel...
1       9.0  Absolutely in love with this game. The learnin...
2      10.0  Ok, I'll be the first to admit, I didn't think...
3      10.0  I dont like football game and races, bat this ...
4      10.0  Psyonix, the developer of Rocket League had pr...
..      ...                                                ...
347    10.0  È molto divertente da giocare con gli amici, p...
348    10.0  I was hesitating about buying this game, and n...
349     8.0  hhohohohoohohohhohohohohoohohohohohohohohohoho...
350     4.0  the concept of the game is not so bad but the ...
351     7.0  • Extremely Fun To Play Solo Or With Friends!•...

[352 rows x 2 columns]
10.0    174
9.0      91
8.0      49
7.0      16
0.0       9
5.0       6
6.0       3
1.0       2
4.0       1
2.0       1
Name: rating, dtype: int64


# 24) Devil May Cry 5

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/devil-may-cry-5/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df46 = pd.DataFrame(review_dict)
    df46 = df46.drop(columns=['name', 'date'])
    df46.to_csv(r"drive/My Drive/csv's/Best/DMC_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df46 = pd.read_csv("drive/My Drive/csv's/Best/DMC_reviews.csv",engine='python' )
df46 = df46.dropna()
del df46['Unnamed: 0']
df46 = df46.reset_index(drop=True)
print(df46)
print(df46['rating'].value_counts())
df46.to_csv(r"drive/My Drive/csv's/Best/DMC_reviews.csv")

     rating                                             review
0      10.0  this game is absolutely worth to wait about 10...
1       9.0  Hideaki Itsuno has been the director to some o...
2       9.0  ok, so i will say this game has a very small a...
3       8.0  I'm no DMC or action game expert (you can read...
4       9.0  What does demons, Japan, metal music, and swor...
..      ...                                                ...
278    10.0  I have been a dmc fan since the first game and...
279    10.0  This is the best fast-peace action game that I...
280     0.0  I never played the game but I figured I'd give...
281     8.0  Nice game play verry enjoe But verry fast to e...
282    10.0  Amazing Hack n slash! Great graphics, people s...

[283 rows x 2 columns]
10.0    131
9.0      69
8.0      37
7.0      12
6.0      10
0.0       6
5.0       6
4.0       5
3.0       4
1.0       2
2.0       1
Name: rating, dtype: int64


# 25) Red Dead Redemption 2

Already have this

# **Worst 25 PS4 User Reviews**

# 1) Madden NFL 21 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/madden-nfl-21/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df47 = pd.DataFrame(review_dict)
    df47 = df47.drop(columns=['name', 'date'])
    df47.to_csv(r"drive/My Drive/csv's/Worst/NFL21_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df47 = pd.read_csv("drive/My Drive/csv's/Worst/NFL21_reviews.csv",engine='python' )
df47 = df47.dropna()
del df47['Unnamed: 0']
df47 = df47.reset_index(drop=True)
print(df47)
print(df47['rating'].value_counts())
df47.to_csv(r"drive/My Drive/csv's/Worst/NFL21_reviews.csv")

     rating                                             review
0       0.0  This game is atrocious. Same game as Madden 20...
1       0.0  An atrocious football simulation game. EA has ...
2       0.0  This game is boring. Full of glitches. Also fr...
3       0.0  This game is straight up unplayable, somehow t...
4       0.0  Same game but actually WORSE. Gameplay is tras...
..      ...                                                ...
257     0.0  The gameplay, the menus, the announcing, it’s ...
258     4.0  Myself and 10 others only buy this game for ou...
259     0.0  Absolutely trash. EA needs to give up their li...
260     2.0  Once again not worth I went against my gut fee...
261     2.0  Same game as last year with no updates to fran...

[262 rows x 2 columns]
0.0     227
1.0      17
2.0       7
4.0       4
3.0       4
10.0      2
5.0       1
Name: rating, dtype: int64


# 2) NBA 2K20

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/nba-2k20/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df48 = pd.DataFrame(review_dict)
    df48 = df48.drop(columns=['name', 'date'])
    df48.to_csv(r"drive/My Drive/csv's/Worst/NBA20_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(7), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6


In [ ]:
df48 = pd.read_csv("drive/My Drive/csv's/Worst/NBA20_reviews.csv",engine='python' )
df48 = df48.dropna()
del df48['Unnamed: 0']
df48 = df48.reset_index(drop=True)
print(df48)
print(df48['rating'].value_counts())
df48.to_csv(r"drive/My Drive/csv's/Worst/NBA20_reviews.csv")

     rating                                             review
0       1.0  Basically made to be a multiplayer game. To ma...
1       1.0  Worst NBA 2K Ever ! Unfinished game with tonns...
2       0.0  This is too much... you buy a game and it is o...
3       0.0  A casino game with a basketball mini game? and...
4       0.0  Trash casino game. Trash casino game. Trash ca...
..      ...                                                ...
552     0.0  It is incredibly disheartening to know 2K is w...
553     0.0  Way too unrealistic and inconsistent, one game...
554     0.0  Not user friendly, worst 2k so far , same neig...
555     0.0  Un simulador más con todos sus juguetitos para...
556     0.0  They ruin the threes.............................

[557 rows x 2 columns]
0.0     429
1.0      43
2.0      22
10.0     18
9.0      10
5.0       8
4.0       8
8.0       7
6.0       5
3.0       4
7.0       3
Name: rating, dtype: int64


# 3) FIFA 20

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/fifa-20/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df49 = pd.DataFrame(review_dict)
    df49 = df49.drop(columns=['name', 'date'])
    df49.to_csv(r"drive/My Drive/csv's/Worst/FIFA20_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(18), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17


In [ ]:
df49 = pd.read_csv("drive/My Drive/csv's/Worst/FIFA20_reviews.csv",engine='python' )
df49 = df49.dropna()
del df49['Unnamed: 0']
df49 = df49.reset_index(drop=True)
print(df49)
print(df49['rating'].value_counts())
df49.to_csv(r"drive/My Drive/csv's/Worst/FIFA20_reviews.csv")

      rating                                             review
0        0.0  I see all these comments saying not to listen ...
1        0.0  EA excels at every game manipulating games and...
2        0.0  This game is totally **** Passes in this game ...
3        0.0  This game hasn't changed since 2017! Just a fe...
4        0.0  The defense changes of fifa 20 are absolutely ...
...      ...                                                ...
1621     0.0  Pokreti igraca su nekordinisani,fizika lopte k...
1622     0.0  Haber si tenemos suerte y por fin desaparece e...
1623    10.0  I think the grades is unfair with this game. I...
1624     8.0  Игра,конечно не без проблем.Есть куча недорабо...
1625     0.0  I was given this game for free by a friend. I ...

[1626 rows x 2 columns]
0.0     1164
1.0      249
10.0      45
2.0       44
3.0       31
5.0       21
8.0       18
4.0       18
6.0       16
7.0       12
9.0        8
Name: rating, dtype: int64


# 4) Star Wars Battlefront II (2017)

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/star-wars-battlefront-ii/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df50 = pd.DataFrame(review_dict)
    df50 = df50.drop(columns=['name', 'date'])
    df50.to_csv(r"drive/My Drive/csv's/Worst/BATTLEFRONTII_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(26), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25


In [ ]:
df50 = pd.read_csv("drive/My Drive/csv's/Worst/BATTLEFRONTII_reviews.csv",engine='python' )
df50 = df50.dropna()
del df50['Unnamed: 0']
df50 = df50.reset_index(drop=True)
print(df50)
print(df50['rating'].value_counts())
df50.to_csv(r"drive/My Drive/csv's/Worst/BATTLEFRONTII_reviews.csv")

      rating                                             review
0        1.0  PLS! ALL! Dont buy this game! If we continue t...
1        0.0  - Preorder bonuses\r- Tiered pricing (Standard...
2        0.0  Thank you EA for the Microtransactions. Thank ...
3        1.0  DICE did a pretty good job with the game in ov...
4        0.0  Pay2Win is an absolute deal killer for me no m...
...      ...                                                ...
2374     7.0                       كل ما ذكرته من طور القصة فقط
2375     0.0  Игра, безусловно, красивая, сюжетка не такая у...
2376     2.0  The Game didn't even tried to be like the old ...
2377     7.0  The story is ok.The gameplay is nice and addic...
2378     6.0  A rocky start to the game, as I was hoping to ...

[2379 rows x 2 columns]
0.0     1694
1.0      171
10.0     132
8.0       82
9.0       75
7.0       46
3.0       45
5.0       35
2.0       34
4.0       33
6.0       32
Name: rating, dtype: int64


# 5) Metal Gear Survive

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/metal-gear-survive/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df51 = pd.DataFrame(review_dict)
    df51 = df51.drop(columns=['name', 'date'])
    df51.to_csv(r"drive/My Drive/csv's/Worst/MGS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df51 = pd.read_csv("drive/My Drive/csv's/Worst/MGS_reviews.csv",engine='python' )
df51 = df51.dropna()
del df51['Unnamed: 0']
df51 = df51.reset_index(drop=True)
print(df51)
print(df51['rating'].value_counts())
df51.to_csv(r"drive/My Drive/csv's/Worst/MGS_reviews.csv")

     rating                                             review
0       0.0  Ripping gamers off! This might be the worst ga...
1       0.0  Micro Transaction for basic game features?!?! ...
2       0.0  "This is like if Nintendo fired Miyamoto and a...
3       3.0  This is not the worst game, however it is a re...
4       0.0  Metal Gear Abortion - a cheap asset flip, game...
..      ...                                                ...
238    10.0                                                .at
239     0.0  just another cash graber game with a lot of mi...
240     6.0  I really wanted to like this game. I have play...
241     0.0                                           F KONAMI
242     5.0  Solo es un juego mas donde lo mas interesante ...

[243 rows x 2 columns]
0.0     159
10.0     28
1.0      14
8.0      11
7.0      10
3.0       8
2.0       4
9.0       3
6.0       3
5.0       2
4.0       1
Name: rating, dtype: int64


# 6) Tony Hawk's Pro Skater 5

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/tony-hawks-pro-skater-5/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df52 = pd.DataFrame(review_dict)
    df52 = df52.drop(columns=['name', 'date'])
    df52.to_csv(r"drive/My Drive/csv's/Worst/THPS5_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df52 = pd.read_csv("drive/My Drive/csv's/Worst/THPS5_reviews.csv",engine='python' )
df52 = df52.dropna()
del df52['Unnamed: 0']
df52 = df52.reset_index(drop=True)
print(df52)
print(df52['rating'].value_counts())
df52.to_csv(r"drive/My Drive/csv's/Worst/THPS5_reviews.csv")

     rating                                             review
0       1.0                            Robomodo strikes again!
1       0.0  How Tony Hawk has fallen from grace. This game...
2       0.0  Wait, so the fix patch for this game is bigger...
3       1.0  It baffles me why major publishers don't do ex...
4       0.0  This game is nothing but digital waste. How an...
..      ...                                                ...
111    10.0  Tony Hawk 5 is the latest and greatest install...
112     3.0  When all is said and done, this game doesn’t d...
113     0.0  Absolutely appalled to see how terrible this g...
114     1.0  This game is the feces that is produced when s...
115     2.0  Лучше бы и не пробовал играть. Графика вырви г...

[116 rows x 2 columns]
0.0     50
1.0     20
2.0     14
3.0      6
7.0      5
8.0      4
10.0     4
6.0      4
9.0      3
5.0      3
4.0      3
Name: rating, dtype: int64


# 7) NFL 20

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/madden-nfl-20/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df53 = pd.DataFrame(review_dict)
    df53 = df53.drop(columns=['name', 'date'])
    df53.to_csv(r"drive/My Drive/csv's/Worst/NFL20_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df53 = pd.read_csv("drive/My Drive/csv's/Worst/NFL20_reviews.csv",engine='python' )
df53 = df53.dropna()
del df53['Unnamed: 0']
df53 = df53.reset_index(drop=True)
print(df53)
print(df53['rating'].value_counts())
df53.to_csv(r"drive/My Drive/csv's/Worst/NFL20_reviews.csv")

     rating                                             review
0       4.0  10 years ago I was dreaming of how amazing Mad...
1       0.0  Grats EA... you hit the pinical of garbage... ...
2       3.0  EA does the bare minimum to get people to thin...
3       2.0  Face of franchise is just a stupid little 2 ga...
4       0.0  Game regressed for the first time ever. Awful ...
..      ...                                                ...
313     0.0  Franchise mode is an exact copy and paste from...
314     0.0                                #FixMaddenFranchise
315     0.0  Fix your bleeping game, quit making excuses an...
316     0.0  Dont buy this game. EA, for many years, has co...
317     0.0  Full of glitches and no depth in franchise mod...

[318 rows x 2 columns]
0.0     195
1.0      50
2.0      21
3.0      13
8.0       9
6.0       6
4.0       6
7.0       5
10.0      5
9.0       5
5.0       3
Name: rating, dtype: int64


# 8) WWE 2K20

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/wwe-2k20/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df54 = pd.DataFrame(review_dict)
    df54 = df54.drop(columns=['name', 'date'])
    df54.to_csv(r"drive/My Drive/csv's/Worst/WWE20_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df54 = pd.read_csv("drive/My Drive/csv's/Worst/WWE20_reviews.csv",engine='python' )
df54 = df54.dropna()
del df54['Unnamed: 0']
df54 = df54.reset_index(drop=True)
print(df54)
print(df54['rating'].value_counts())
df54.to_csv(r"drive/My Drive/csv's/Worst/WWE20_reviews.csv")

     rating                                             review
0       0.0  I'm not surprised this game is a mess - Yukes ...
1       0.0  This game is a huge step backwards. I love wre...
2       0.0  Beyond broken. Only token things added to this...
3       0.0  the worst game of the entire series, the game ...
4       0.0  The game is literally unplayable. Tons of glit...
..      ...                                                ...
125     3.0  Игру не покупал, но мой друг Максим (кстати у ...
126     2.0  In my opinion. I have played WWE games since 2...
127     1.0  I've been playing Smackdown vs Raw and  their ...
128     1.0  After an impressive showing with WWE 2K19, WWE...
129     4.0  Sadly, Choosing to part ways with the old deve...

[130 rows x 2 columns]
0.0     69
1.0     20
2.0      9
4.0      9
8.0      5
5.0      5
10.0     4
3.0      4
9.0      2
7.0      2
6.0      1
Name: rating, dtype: int64


# 9) NBA 2K18

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/nba-2k18/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df55 = pd.DataFrame(review_dict)
    df55 = df55.drop(columns=['name', 'date'])
    df55.to_csv(r"drive/My Drive/csv's/Worst/NBA18_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df55 = pd.read_csv("drive/My Drive/csv's/Worst/NBA18_reviews.csv",engine='python' )
df55 = df55.dropna()
del df55['Unnamed: 0']
df55 = df55.reset_index(drop=True)
print(df55)
print(df55['rating'].value_counts())
df55.to_csv(r"drive/My Drive/csv's/Worst/NBA18_reviews.csv")

     rating                                             review
0       0.0  Microtransactions don't belong in fully priced...
1       0.0  Seriously, Im Canadian, i paid 79.99 + tax for...
2       4.0  The last few years NBA 2K's were one thing but...
3       1.0    You need a lot of VC to upgrade your attributes
4       0.0  F2P game mechanics in a "AAA" title for 60$ fo...
..      ...                                                ...
235     0.0  Microtransactions!!\rstay away from this infin...
236     0.0  Damn. Critics are really getting paid to swall...
237     0.0  Terrible game, no improvements and terrible an...
238     2.0  Esse é o pior jogo de basquete que já joguei e...
239     0.0  User ratings are always better than critic rat...

[240 rows x 2 columns]
0.0     134
1.0      41
2.0      12
3.0      11
4.0      10
8.0       8
5.0       7
10.0      5
6.0       4
9.0       4
7.0       4
Name: rating, dtype: int64


# 10) FIFA 19

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/fifa-19/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df56 = pd.DataFrame(review_dict)
    df56 = df56.drop(columns=['name', 'date'])
    df56.to_csv(r"drive/My Drive/csv's/Worst/FIFA19_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df56 = pd.read_csv("drive/My Drive/csv's/Worst/FIFA19_reviews.csv",engine='python' )
df56 = df56.dropna()
del df56['Unnamed: 0']
df56 = df56.reset_index(drop=True)
print(df56)
print(df56['rating'].value_counts())
df56.to_csv(r"drive/My Drive/csv's/Worst/FIFA19_reviews.csv")

     rating                                             review
0       1.0  if you drop your joystick the game play alone ...
1       0.0    Fifa 19 Ping-pong edition with change colour...
2       0.0  This game is trash.I don't know from where to ...
3       0.0  Embarasing, nothing else, just embarasing, Emb...
4       0.0  How could you possibly justify selling the exa...
..      ...                                                ...
359     3.0  There shouldn't be a percentage requirement to...
360     2.0  The game is just like 18 and 17, the focus is ...
361     0.0  O pior fifa que já joguei na minha vida, cheio...
362     8.0  hhohohohoohohohhohohohohoohohohohohohohohohoho...
363     5.0  التقييم ينطق ليس تقييمي فحسب بل تققيم الناس لل...

[364 rows x 2 columns]
0.0     202
1.0      55
2.0      20
8.0      18
10.0     14
4.0      14
5.0      12
6.0       9
3.0       9
7.0       6
9.0       5
Name: rating, dtype: int64


# 11) NFL 19

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/madden-nfl-19/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df57 = pd.DataFrame(review_dict)
    df57 = df57.drop(columns=['name', 'date'])
    df57.to_csv(r"drive/My Drive/csv's/Worst/NFL19_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df57 = pd.read_csv("drive/My Drive/csv's/Worst/NFL19_reviews.csv",engine='python' )
df57 = df57.dropna()
del df57['Unnamed: 0']
df57 = df57.reset_index(drop=True)
print(df57)
print(df57['rating'].value_counts())
df57.to_csv(r"drive/My Drive/csv's/Worst/NFL19_reviews.csv")

     rating                                             review
0       2.0  I'll state this right off the bat: I'm someone...
1       0.0  Another game used by EA as a financial pawn to...
2       0.0  I decided to get my old GameCube out and play ...
3       4.0  Probably the last game I'll purchase from EA S...
4       0.0  This game is simply trash. NFL doesn’t give a ...
..      ...                                                ...
117     0.0  I reset my ps4 because the game froze everythi...
118     5.0  There is one single reason to buy this product...
119     0.0  Absolutely garbage  plz do not buy.    I hope ...
120     1.0  Just think copy and paste from previous years ...
121     2.0  There charging AAA prices for a crap game. Mad...

[122 rows x 2 columns]
0.0     63
2.0     16
1.0     14
4.0      8
5.0      6
8.0      3
6.0      3
3.0      3
10.0     2
9.0      2
7.0      2
Name: rating, dtype: int64


# 12) Battlefield V

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/battlefield-v/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df58 = pd.DataFrame(review_dict)
    df58 = df58.drop(columns=['name', 'date'])
    df58.to_csv(r"drive/My Drive/csv's/Worst/BFV_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df58 = pd.read_csv("drive/My Drive/csv's/Worst/BFV_reviews.csv",engine='python' )
df58 = df58.dropna()
del df58['Unnamed: 0']
df58 = df58.reset_index(drop=True)
print(df58)
print(df58['rating'].value_counts())
df58.to_csv(r"drive/My Drive/csv's/Worst/BFV_reviews.csv")

     rating                                             review
0       0.0  В игре про Вторую мировую войну нету страны, к...
1       0.0  t has a lot of bugs, not completed story mode ...
2       0.0  Лучше поиграть в Red Orchestra 2: Heroes of St...
3       0.0  Waiting for EA to close DICE and they will sto...
4       0.0  Developer said not to buy it.I totaly agree.It...
..      ...                                                ...
765     0.0  Стрельба в Хойке лучше чем в этом кале.Очень м...
766     2.0  Worst battlefield ever hardline was bad but om...
767     1.0  Как можно оценить игру в которой действия прои...
768     4.0  As peculiaridades da Dice continuam ali. Quand...
769     0.0  This game has its ups and downs. It is an alri...

[770 rows x 2 columns]
0.0     377
1.0      93
10.0     60
2.0      41
9.0      34
5.0      33
8.0      33
4.0      29
6.0      26
7.0      23
3.0      21
Name: rating, dtype: int64


# 13) NBA 2K19

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/nba-2k19/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df59 = pd.DataFrame(review_dict)
    df59 = df59.drop(columns=['name', 'date'])
    df59.to_csv(r"drive/My Drive/csv's/Worst/NBA19_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df59 = pd.read_csv("drive/My Drive/csv's/Worst/NBA19_reviews.csv",engine='python' )
df59 = df59.dropna()
del df59['Unnamed: 0']
df59 = df59.reset_index(drop=True)
print(df59)
print(df59['rating'].value_counts())
df59.to_csv(r"drive/My Drive/csv's/Worst/NBA19_reviews.csv")

     rating                                             review
0       4.0  EDIT: Changing score to 6 after 3 full days of...
1       1.0  I’ve been playing NBA 2K since 2k3 and I alway...
2       0.0  Buggy mess full of microtransactions. Every ye...
3       2.0  Paid for the 20th Year Anniversary edition and...
4       5.0  Good: Seem to earn a bit more VC from games, H...
..      ...                                                ...
138    10.0  It's amazing to see how bad NBA 2K20 was for u...
139     4.0  Está cada vez pior gostar de basquete e de vid...
140     4.0  Servers are garbage as always. Ridiculous grin...
141     0.0  I do not found nothing new in this "new" arcad...
142     0.0  This thing is so scripted and so unfair when y...

[143 rows x 2 columns]
0.0     70
1.0     24
8.0     11
4.0     10
2.0      9
10.0     7
6.0      3
3.0      3
5.0      3
9.0      2
7.0      1
Name: rating, dtype: int64


# 14) Fallout 76

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/fallout-76/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df60 = pd.DataFrame(review_dict)
    df60 = df60.drop(columns=['name', 'date'])
    df60.to_csv(r"drive/My Drive/csv's/Worst/FALLOUT76_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(17), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16


In [ ]:
df60 = pd.read_csv("drive/My Drive/csv's/Worst/FALLOUT76_reviews.csv",engine='python' )
df60 = df60.dropna()
del df60['Unnamed: 0']
df60 = df60.reset_index(drop=True)
print(df60)
print(df60['rating'].value_counts())
df60.to_csv(r"drive/My Drive/csv's/Worst/FALLOUT76_reviews.csv")

      rating                                             review
0        4.0  The team at Bethesda have developed an experie...
1        0.0  My god, Bethesda... what have you done?\rGraph...
2        0.0  Today i fnished the first "Dark Souls" and jus...
3        0.0  The franchise is no longer an RPG.  this game ...
4        2.0  While it sounds interesting on paper, in pract...
...      ...                                                ...
1565     1.0  This game was In my opinion looked so cool on ...
1566    10.0  Отличное ММО , длс с людьми , новые квесты и с...
1567     0.0  Do i even need to say why?\r It's fallout 76, ...
1568     0.0  квесты не работают. Собрал кровь гуля, закинул...
1569     7.0  The game is fun, and looks great. But still ha...

[1570 rows x 2 columns]
0.0     693
10.0    186
1.0     165
9.0     115
8.0     103
2.0      95
3.0      50
7.0      47
4.0      47
5.0      40
6.0      29
Name: rating, dtype: int64


# 15) Modern Warfare (Extracted again for more recent reviews)

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-modern-warfare/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df61 = pd.DataFrame(review_dict)
    df61 = df61.drop(columns=['name', 'date'])
    df61.to_csv(r"drive/My Drive/csv's/Worst/MW_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(44), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8
[*] Working on page: 9
[*] Working on page: 10
[*] Working on page: 11
[*] Working on page: 12
[*] Working on page: 13
[*] Working on page: 14
[*] Working on page: 15
[*] Working on page: 16
[*] Working on page: 17
[*] Working on page: 18
[*] Working on page: 19
[*] Working on page: 20
[*] Working on page: 21
[*] Working on page: 22
[*] Working on page: 23
[*] Working on page: 24
[*] Working on page: 25
[*] Working on page: 26
[*] Working on page: 27
[*] Working on page: 28
[*] Working on page: 29
[*] Working on page: 30
[*] Working on page: 31
[*] Working on page: 32
[*] Working on page: 33
[*] Working on page: 34
[*] Working on page: 35
[*] Working on page: 36
[*] Working on page: 37
[*] Working on page: 38
[*] Working on page: 39
[*] Working on page: 40
[*] Working on page: 41
[*

In [ ]:
df61 = pd.read_csv("drive/My Drive/csv's/Worst/MW_reviews.csv",engine='python' )
df61 = df61.dropna()
del df61['Unnamed: 0']
df61 = df61.reset_index(drop=True)
print(df61)
print(df61['rating'].value_counts())
df61.to_csv(r"drive/My Drive/csv's/Worst/MW_reviews.csv")

      rating                                             review
0        0.0  The developers just decided to accuse the Russ...
1        0.0  This game is nothing but a russophobic propaga...
2        0.0  I even created a Metacritic account for the re...
3        0.0  This **** Propaganda. You think we buy this ga...
4        0.0  Russophobic and hate-mongering game. Authors s...
...      ...                                                ...
3923     0.0  **** server for mp , lag comp , **** hit marke...
3924     5.0  Would have totally loved to kill some russian!...
3925     0.0  I never thought I’d ever give a game a zero. I...
3926     8.0  when the game first came out it didn't win me ...
3927     8.0  I thoroughly enjoyed this entry in the Call of...

[3928 rows x 2 columns]
0.0     2020
10.0     814
1.0      357
9.0      185
8.0      140
2.0      121
3.0       68
7.0       67
4.0       66
6.0       47
5.0       43
Name: rating, dtype: int64


# 16) Anthem

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/anthem/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df62 = pd.DataFrame(review_dict)
    df62 = df62.drop(columns=['name', 'date'])
    df62.to_csv(r"drive/My Drive/csv's/Worst/ANTHEM_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df62 = pd.read_csv("drive/My Drive/csv's/Worst/ANTHEM_reviews.csv",engine='python' )
df62 = df62.dropna()
del df62['Unnamed: 0']
df62 = df62.reset_index(drop=True)
print(df62)
print(df62['rating'].value_counts())
df62.to_csv(r"drive/My Drive/csv's/Worst/ANTHEM_reviews.csv")

     rating                                             review
0       4.0  There is some fun to be had here but so many c...
1       4.0  Its not a 0 but it is definitely not a 10 eith...
2       0.0  12 hours into the game and the only thing Anth...
3       1.0  One thing that won't be "fixed in future": the...
4       3.0  jogo mal feito.vale no maximo 50 e pronto.EA s...
..      ...                                                ...
381     4.0  The game is missing a story line, or at least ...
382     0.0  I expect that I can play a game for which I ha...
383     0.0  This game has so much potential! It clearly fe...
384     0.0  esse jogo quase queimou  meu ps4 , cheio de bu...
385     0.0  Anthem is painful. Everything is so slow, so b...

[386 rows x 2 columns]
0.0     110
10.0     60
8.0      44
2.0      31
9.0      27
3.0      27
4.0      27
1.0      26
6.0      13
7.0      12
5.0       9
Name: rating, dtype: int64


# 17) FIFA 18 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/fifa-18/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df63 = pd.DataFrame(review_dict)
    df63 = df63.drop(columns=['name', 'date'])
    df63.to_csv(r"drive/My Drive/csv's/Worst/FIFA18_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df63 = pd.read_csv("drive/My Drive/csv's/Worst/FIFA18_reviews.csv",engine='python' )
df63 = df63.dropna()
del df63['Unnamed: 0']
df63 = df63.reset_index(drop=True)
print(df63)
print(df63['rating'].value_counts())
df63.to_csv(r"drive/My Drive/csv's/Worst/FIFA18_reviews.csv")

     rating                                             review
0       0.0  I must admit SCRIPTING is shamelessly there, y...
1       0.0  loot boxes and pay to win do not buy keep your...
2       0.0  Сделать честную игру!Это не киберспорт!!!Сообщ...
3       0.0  Mas de lo mismo una y otra vez sacando dinero ...
4       0.0  nu recomand acest joc , e prea ofticat :)))) o...
..      ...                                                ...
178     6.0  Gráficos decentes, es agradable con amigos, no...
179     7.0  Igual que los demás, pero un poco mejor, los s...
180     7.0  7/10------------------------------------------...
181     0.0  They make a **** of optimization for this game...
182     4.0                         لعبة حتى القرود تأبى لعبها

[183 rows x 2 columns]
0.0     59
8.0     27
1.0     18
10.0    15
6.0     14
7.0     12
5.0      9
4.0      9
9.0      8
3.0      8
2.0      4
Name: rating, dtype: int64


# 18) Street Fighter V

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/street-fighter-v/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df64 = pd.DataFrame(review_dict)
    df64 = df64.drop(columns=['name', 'date'])
    df64.to_csv(r"drive/My Drive/csv's/Worst/SFV_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(6), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5


In [ ]:
df64 = pd.read_csv("drive/My Drive/csv's/Worst/SFV_reviews.csv",engine='python' )
df64 = df64.dropna()
del df64['Unnamed: 0']
df64 = df64.reset_index(drop=True)
print(df64)
print(df64['rating'].value_counts())
df64.to_csv(r"drive/My Drive/csv's/Worst/SFV_reviews.csv")

     rating                                             review
0       2.0  The game is not completed. Feels like its a de...
1       2.0  sad sad sad... Street Fighter doesnt deserve t...
2       2.0  I stayed up until the midnight release... I ha...
3       2.0  most multiplayer games come out like this. Alw...
4       0.0  English is not my native language , but that d...
..      ...                                                ...
442     0.0  So, you literally just buy game but, you dont ...
443     0.0  La más grande estafa de los últimos 20 años. N...
444     1.0  Increible que capcom venda un juego incompleto...
445     7.0  A terrible launch, but a great improvement ove...
446     6.0  Look. The game isn't as terrible as everyone s...

[447 rows x 2 columns]
0.0     144
1.0      60
10.0     42
4.0      38
2.0      34
3.0      29
8.0      23
6.0      23
9.0      22
5.0      17
7.0      15
Name: rating, dtype: int64


# 19) Fortnite

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/fortnite/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df65 = pd.DataFrame(review_dict)
    df65 = df65.drop(columns=['name', 'date'])
    df65.to_csv(r"drive/My Drive/csv's/Worst/FORTNITE_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df65 = pd.read_csv("drive/My Drive/csv's/Worst/FORTNITE_reviews.csv",engine='python' )
df65 = df65.dropna()
del df65['Unnamed: 0']
df65 = df65.reset_index(drop=True)
print(df65)
print(df65['rating'].value_counts())
df65.to_csv(r"drive/My Drive/csv's/Worst/FORTNITE_reviews.csv")

     rating                                             review
0       1.0  Short: Save your money!\r This game has Potent...
1       1.0  I can't recommend this game and the behavior o...
2       0.0                                 Pay to Pay to win.
3      10.0  Super Addictive Game. Played more than 20 Hour...
4       5.0  Gameplay is solid.  Was having lots of fun.  K...
..      ...                                                ...
271    10.0  este jogo é muito bom, foi o melhor jogo onlin...
272     0.0  TRASH Same **** every round, only little 7 yea...
273     0.0  this game is the worst game in the history of ...
274     1.0              Gramnda didn gave me gaem for crimmas
275     0.0  Einfach gesagt das spiel ist Scheiße schlechte...

[276 rows x 2 columns]
0.0     111
1.0      25
10.0     21
8.0      20
9.0      18
6.0      15
4.0      15
5.0      14
2.0      13
7.0      12
3.0      12
Name: rating, dtype: int64


# 20) Mortal Kombat 11

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/mortal-kombat-11/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df66 = pd.DataFrame(review_dict)
    df66 = df66.drop(columns=['name', 'date'])
    df66.to_csv(r"drive/My Drive/csv's/Worst/MK11_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(9), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6
[*] Working on page: 7
[*] Working on page: 8


In [ ]:
df66 = pd.read_csv("drive/My Drive/csv's/Worst/MK11_reviews.csv",engine='python' )
df66 = df66.dropna()
del df66['Unnamed: 0']
df66 = df66.reset_index(drop=True)
print(df66)
print(df66['rating'].value_counts())
df66.to_csv(r"drive/My Drive/csv's/Worst/MK11_reviews.csv")

     rating                                             review
0       1.0  Оптимизации нет, добавили кучу ненужного донат...
1       0.0  Always online single player game with microSca...
2       4.0  You done **** it up ! What the hell ? All this...
3       0.0  The game is ok. Good graphics, sound and gamep...
4       0.0  6000$ for unlocking all? Hahahah no, thanks, W...
..      ...                                                ...
745     7.0  First, the good aspects of this game: Mortal K...
746     9.0  As of now, this is peak Mortal Kombat, and lik...
747     6.0  In my opinion this is the worst mortal kombat ...
748     9.0  Şu zamana kadar çıkmış en iyi ''Mortal Kombat'...
749     7.0  Una buena historia, buenos personajes, buena j...

[750 rows x 2 columns]
0.0     302
10.0    101
1.0      90
9.0      51
2.0      50
8.0      35
3.0      35
4.0      31
5.0      20
6.0      19
7.0      16
Name: rating, dtype: int64


# 21) Call of Duty Ghosts

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-ghosts/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df67 = pd.DataFrame(review_dict)
    df67 = df67.drop(columns=['name', 'date'])
    df67.to_csv(r"drive/My Drive/csv's/Worst/GHOSTS_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df67 = pd.read_csv("drive/My Drive/csv's/Worst/GHOSTS_reviews.csv",engine='python' )
df67 = df67.dropna()
del df67['Unnamed: 0']
df67 = df67.reset_index(drop=True)
print(df67)
print(df67['rating'].value_counts())
df67.to_csv(r"drive/My Drive/csv's/Worst/GHOSTS_reviews.csv")

     rating                                             review
0       5.0  Why can't I learn my lesson? Every year its th...
1       2.0  The sad truth is, Call of Duty campaigns  and ...
2       1.0  If you play any Call of Duty, you've played GH...
3       2.0  rinse and repeat vol. 3000 so rinse and repeat...
4       1.0  Been a CoD for since CoD2 (not MW2). This game...
..      ...                                                ...
344     3.0  Bored, generic, repetitive, a bad story and a ...
345     6.0  Ghosts isn't a bad game; it just isn't special...
346     9.0  This game is an Activision game and i can say ...
347     7.0  Tiene muy mala fama pero para mi es uno de los...
348    10.0  fjejdukkekxxekkxnexbezgwyasnswxkuexyzvgwzbqtbz...

[349 rows x 2 columns]
0.0     61
10.0    43
8.0     36
6.0     30
7.0     30
5.0     30
4.0     28
2.0     25
3.0     22
9.0     22
1.0     22
Name: rating, dtype: int64


# 22) Call of Duty: Infinite Warfare

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-infinite-warfare/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df68 = pd.DataFrame(review_dict)
    df68 = df68.drop(columns=['name', 'date'])
    df68.to_csv(r"drive/My Drive/csv's/Worst/IW_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(5), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4


In [ ]:
df68 = pd.read_csv("drive/My Drive/csv's/Worst/IW_reviews.csv",engine='python' )
df68 = df68.dropna()
del df68['Unnamed: 0']
df68 = df68.reset_index(drop=True)
print(df68)
print(df68['rating'].value_counts())
df68.to_csv(r"drive/My Drive/csv's/Worst/IW_reviews.csv")

     rating                                             review
0       5.0  Campaign is okay. JUST okay. I liked the visua...
1       6.0  Here's the deal. COD IW is not a bad game. But...
2       7.0  Story/Campaign: Thrilling dogfights, excellent...
3       3.0  I couldn't be more livid. The lad who said thi...
4       0.0  Shameless copy paste of blops 3, once more wit...
..      ...                                                ...
376    10.0  I will note cod iw on one factor, the campaign...
377     6.0  El peor cod que jugue, repetitivo como siempre...
378     3.0  I know this game gets alot of hate, and I can ...
379    10.0  Hands down this is the best campaign out of an...
380     8.0  El juego en sí es muy bueno, pero tiene sus pe...

[381 rows x 2 columns]
0.0     85
10.0    65
8.0     40
5.0     31
1.0     30
9.0     28
7.0     25
4.0     22
3.0     22
6.0     17
2.0     16
Name: rating, dtype: int64


# 23) Call of Duty: Black Ops 4 

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/call-of-duty-black-ops-4/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df69 = pd.DataFrame(review_dict)
    df69 = df69.drop(columns=['name', 'date'])
    df69.to_csv(r"drive/My Drive/csv's/Worst/BO4_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(7), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3
[*] Working on page: 4
[*] Working on page: 5
[*] Working on page: 6


In [ ]:
df69 = pd.read_csv("drive/My Drive/csv's/Worst/BO4_reviews.csv",engine='python' )
df69 = df69.dropna()
del df69['Unnamed: 0']
df69 = df69.reset_index(drop=True)
print(df69)
print(df69['rating'].value_counts())
df69.to_csv(r"drive/My Drive/csv's/Worst/BO4_reviews.csv")

     rating                                             review
0       4.0  The removal of a single-player campaign is ind...
1       0.0  No campaign, garbage and a waste of my time. T...
2       0.0  Ever notice why Infinity Ward games had a bett...
3       1.0  Why are people who praise this game taking sho...
4       2.0  COD BO has always has the best COD campaigns. ...
..      ...                                                ...
499     6.0  Un buen juego que ha logrado recuperar mi inte...
500     6.0  Would be higher if the micro transaction syste...
501     0.0  This game is garbage. Terribly long, and borin...
502     4.0  I hated this game initially, but overtime, i s...
503     7.0  عبث باللاعبين واستغلال حبهم لسلسلة من أجل حلبه...

[504 rows x 2 columns]
0.0     158
10.0     68
1.0      51
8.0      43
2.0      32
9.0      30
7.0      29
5.0      28
3.0      27
4.0      20
6.0      18
Name: rating, dtype: int64


# 24) Far Cry: New Dawn

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/far-cry-new-dawn/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df70 = pd.DataFrame(review_dict)
    df70 = df70.drop(columns=['name', 'date'])
    df70.to_csv(r"drive/My Drive/csv's/Worst/FCND_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(3), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2


In [ ]:
df70 = pd.read_csv("drive/My Drive/csv's/Worst/FCND_reviews.csv",engine='python' )
df70 = df70.dropna()
del df70['Unnamed: 0']
df70 = df70.reset_index(drop=True)
print(df70)
print(df70['rating'].value_counts())
df70.to_csv(r"drive/My Drive/csv's/Worst/FCND_reviews.csv")

     rating                                             review
0       0.0  Easy cash-in for Ubisoft before the inevitable...
1       0.0  Re-used Far Cry 5 locations with sunny-rainbow...
2       2.0  This production is a really shame...is trash.....
3       0.0  Это глупая, отвратительная игра на базе дальни...
4       0.0  Whoever is directing this train-wreck needs to...
..      ...                                                ...
139     8.0  One of the action games which try to be more R...
140     0.0  В начале игры нету возможности играть по стелс...
141     7.0  Not the best Far Cry but still slightly better...
142     8.0  As long as you understand what you're getting ...
143     0.0  Uma porcaria, os mesmos bots bugs do Far Cry 5...

[144 rows x 2 columns]
0.0     36
10.0    22
8.0     19
7.0     15
2.0     10
4.0      9
6.0      9
9.0      8
1.0      6
3.0      6
5.0      4
Name: rating, dtype: int64


# 25) Need for Speed: Payback

In [ ]:
def scrap_page(page, review_dict):
    url = 'https://www.metacritic.com/game/playstation-4/need-for-speed-payback/user-reviews?page=' + str(page)
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=user_agent)

    soup = BeautifulSoup(response.text, 'html.parser')
    for review in soup.find_all('div', class_='review_content'):
        if review.find('div', class_='name') == None:
            break
        review_dict['name'].append(review.find('div', class_='name').find('a').text)
        review_dict['date'].append(review.find('div', class_='date').text)
        review_dict['rating'].append(review.find('div', class_='review_grade').find_all('div')[0].text)
        if review.find('span', class_='blurb blurb_expanded'):
            review_dict['review'].append(
                review.find('span', class_='blurb blurb_expanded').text if review.find('span',
                                                                                       class_='blurb blurb_expanded') else None)
        else:
            review_dict['review'].append(
                review.find('div', class_='review_body').find('span').text if review.find('div',
                                                                                          class_='review_body').find(
                    'span') else None)


def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    review_dict = {'name': [], 'date': [], 'rating': [], 'review': []}
    for i in out:
        thread_list = []
        for page in i:
            print('[*] Working on page: %d' % page)
            thread = threading.Thread(target=scrap_page, args=(page, review_dict,))
            thread_list.append(thread)
            thread.start()
        for t in thread_list:
            t.join()
    df71 = pd.DataFrame(review_dict)
    df71 = df71.drop(columns=['name', 'date'])
    df71.to_csv(r"drive/My Drive/csv's/Worst/NFSP_reviews.csv")


if __name__ == '__main__':
    
    chunkIt(range(4), 100)

[*] Working on page: 0
[*] Working on page: 1
[*] Working on page: 2
[*] Working on page: 3


In [ ]:
df71 = pd.read_csv("drive/My Drive/csv's/Worst/NFSP_reviews.csv",engine='python' )
df71 = df71.dropna()
del df71['Unnamed: 0']
df71 = df71.reset_index(drop=True)
print(df71)
print(df71['rating'].value_counts())
df71.to_csv(r"drive/My Drive/csv's/Worst/NFSP_reviews.csv")

     rating                                             review
0       1.0  EA ruins another game with a loot box progress...
1       4.0  Its not the worst entry, however its obvious t...
2       0.0  Just disappointed EA. Pumping the machine with...
3       0.0  Another lootboxes title which gives you 2 choi...
4       0.0  The cool stuff, like the fast & furious kind o...
..      ...                                                ...
222     8.0  Вполне адекватный NFS, к управлению привыкаешь...
223     2.0  EA is a disaster of gaming with their lutbox, ...
224     2.0  Nothing to say more about than the gameplay is...
225     2.0  Interesting story. Well, interesting enough fo...
226     1.0  Very very bad. Needfor speed 2015 was better t...

[227 rows x 2 columns]
0.0     85
1.0     25
10.0    23
8.0     18
9.0     16
2.0     14
4.0     14
3.0     11
7.0      8
5.0      7
6.0      6
Name: rating, dtype: int64
